# Install Dependencies

In [1]:
import torch
import time
import json
import nltk
import os
import platform
import psutil
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt', quiet=True)

# Define fixed test cases
test_prompts = [
    {"id": 1, "prompt": "Explain how neural networks learn.", "max_new_tokens": 80},
    {"id": 2, "prompt": "Write a haiku about machine learning.", "max_new_tokens": 60},
    {"id": 3, "prompt": "What is quantization in deep learning?", "max_new_tokens": 100}
]

def get_system_specs():
    """Get system specifications"""
    specs = {
        "os": f"{platform.system()} {platform.release()}",
        "processor": platform.processor(),
        "cpu_cores_physical": psutil.cpu_count(logical=False),
        "cpu_cores_logical": psutil.cpu_count(logical=True),
        "ram_total_gb": round(psutil.virtual_memory().total / (1024**3), 2),
        "ram_available_gb": round(psutil.virtual_memory().available / (1024**3), 2),
        "python_version": platform.python_version(),
        "pytorch_version": torch.__version__,
        "cuda_available": torch.cuda.is_available(),
        "cuda_version": torch.version.cuda if torch.cuda.is_available() else None,
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None
    }
    return specs

def get_model_size_mb(model):
    """Calculate model size in MB"""
    param_size = 0
    buffer_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_mb = (param_size + buffer_size) / (1024**2)
    return round(size_mb, 2)

def get_model_params(model):
    """Get total parameters count"""
    return sum(p.numel() for p in model.parameters())

# Get and display system specs
system_specs = get_system_specs()
print("🖥️ SYSTEM SPECIFICATIONS")
print("="*50)
for key, value in system_specs.items():
    print(f"   {key}: {value}")

🖥️ SYSTEM SPECIFICATIONS
   os: Windows 10
   processor: AMD64 Family 23 Model 17 Stepping 0, AuthenticAMD
   cpu_cores_physical: 4
   cpu_cores_logical: 8
   ram_total_gb: 13.67
   ram_available_gb: 1.57
   python_version: 3.12.4
   pytorch_version: 2.5.1+cpu
   cuda_available: False
   cuda_version: None
   gpu_name: None


In [3]:
test_prompts

[{'id': 1,
  'prompt': 'Explain how neural networks learn.',
  'max_new_tokens': 80},
 {'id': 2,
  'prompt': 'Write a haiku about machine learning.',
  'max_new_tokens': 60},
 {'id': 3,
  'prompt': 'What is quantization in deep learning?',
  'max_new_tokens': 100}]

**Load Tokenizer & FP16 Model CPU Only**

In [3]:
model_id = "Qwen/Qwen2-1.5B-Instruct"

print("⏳ Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("⏳ Loading FP16 model on CPU (this will take 2-5 mins)...")
# Load model in float16 but place on CPU to avoid OOM
model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map={"": "cpu"},  # Force CPU
    low_cpu_mem_usage=True
)
print("✅ FP16 model loaded on CPU.")

# Get FP16 model size
fp16_size_mb = get_model_size_mb(model_fp16)
fp16_params = get_model_params(model_fp16)
print(f"\n📏 FP16 Model Size: {fp16_size_mb} MB")
print(f"📊 Total Parameters: {fp16_params:,}")

⏳ Loading tokenizer...
⏳ Loading FP16 model on CPU (this will take 2-5 mins)...
✅ FP16 model loaded on CPU.

📏 FP16 Model Size: 2944.41 MB
📊 Total Parameters: 1,543,714,304


**Function to generate text and measure time**

In [4]:
def generate_text(model, tokenizer, prompt, max_new_tokens=100, device="cpu"):
    # Format prompt for Mistral
    formatted_prompt = f"[INST] {prompt} [/INST]"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    start = time.time()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    latency = time.time() - start

    generated = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    num_tokens = outputs.shape[1] - inputs["input_ids"].shape[1]

    return {
        "text": generated.strip(),
        "latency": round(latency, 3),
        "tokens": num_tokens,
        "tokens_per_sec": round(num_tokens / latency, 2)
    }

**FP16 (baseline) inference**

In [5]:
print("🔵 Running FP16 (baseline) inference on CPU...")
baseline_results = []

for case in test_prompts:
    print(f"  Prompt {case['id']}: '{case['prompt']}'")
    result = generate_text(model_fp16, tokenizer, case["prompt"], case["max_new_tokens"], device="cpu")
    result["id"] = case["id"]
    baseline_results.append(result)
    print(f"    → {result['latency']}s | {result['tokens_per_sec']} tok/s")

# Delete FP16 model to free RAM
del model_fp16
torch.cuda.empty_cache()

🔵 Running FP16 (baseline) inference on CPU...
  Prompt 1: 'Explain how neural networks learn.'
    → 438.287s | 0.18 tok/s
  Prompt 2: 'Write a haiku about machine learning.'
    → 345.274s | 0.17 tok/s
  Prompt 3: 'What is quantization in deep learning?'
    → 537.283s | 0.19 tok/s


In [6]:
print("⚙️ Applying INT8 dynamic quantization (PyTorch native)...")

# Since bitsandbytes requires CUDA on Windows and CUDA is not available,
# we'll use PyTorch's native dynamic quantization for CPU

# Reload the model for quantization (since we deleted fp16 model)
model_for_quant = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,  # Load in FP32 for quantization
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Get FP32 model size before quantization
fp32_size_mb = get_model_size_mb(model_for_quant)
print(f"📏 FP32 Model Size (before quantization): {fp32_size_mb} MB")

# Apply dynamic quantization to Linear layers
model_int8 = torch.quantization.quantize_dynamic(
    model_for_quant,
    {torch.nn.Linear},  # Quantize only Linear layers
    dtype=torch.qint8
)

# Get INT8 model size after quantization
int8_size_mb = get_model_size_mb(model_int8)
int8_params = get_model_params(model_int8)
compression_ratio = fp32_size_mb / int8_size_mb if int8_size_mb > 0 else 0

print(f"📏 INT8 Model Size (after quantization): {int8_size_mb} MB")
print(f"📉 Compression Ratio: {compression_ratio:.2f}x")
print(f"💾 Memory Saved: {fp32_size_mb - int8_size_mb:.2f} MB")

# Clean up the original model
del model_for_quant
torch.cuda.empty_cache()

print("✅ INT8 dynamically quantized model ready.")

⚙️ Applying INT8 dynamic quantization (PyTorch native)...
📏 FP32 Model Size (before quantization): 5888.81 MB
📏 INT8 Model Size (after quantization): 890.59 MB
📉 Compression Ratio: 6.61x
💾 Memory Saved: 4998.22 MB
✅ INT8 dynamically quantized model ready.


In [7]:
device = "cpu"  # INT8 quantization runs on CPU
print(f"🟢 Running INT8 inference on {device.upper()}...")

quant_results = []
for case in test_prompts:
    print(f"  Prompt {case['id']}: '{case['prompt']}'")
    result = generate_text(model_int8, tokenizer, case["prompt"], case["max_new_tokens"], device=device)
    result["id"] = case["id"]
    quant_results.append(result)
    print(f"    → {result['latency']}s | {result['tokens_per_sec']} tok/s")

🟢 Running INT8 inference on CPU...
  Prompt 1: 'Explain how neural networks learn.'
    → 19.3s | 4.15 tok/s
  Prompt 2: 'Write a haiku about machine learning.'
    → 14.906s | 4.03 tok/s
  Prompt 3: 'What is quantization in deep learning?'
    → 26.314s | 3.8 tok/s


# 🔷 INT4 Quantization (4-bit)

INT4 quantization pushes compression further by using only 4 bits per weight.

## How INT4 Works:
1. **Weight Grouping**: Weights are divided into groups (32-128 weights)
2. **Per-group Scale**: Each group has its own scale factor
3. **Mapping**: Values mapped to range [-8, 7] or [0, 15]

$$x_{int4} = \text{round}\left(\frac{x}{\text{scale}}\right)$$

## Benefits:
- **8x compression** vs FP32
- **4x compression** vs FP16
- Ideal for edge devices with limited RAM

In [ ]:
# ============================================================================
# INT4 QUANTIZATION IMPLEMENTATION
# ============================================================================
# Since BitsAndBytes requires CUDA, we'll implement a manual INT4 quantization
# that works on CPU using PyTorch

def quantize_tensor_to_int4(tensor, group_size=32):
    """
    Quantize a tensor to INT4 format using per-group quantization.
    
    How it works:
    1. Reshape tensor into groups
    2. Calculate scale and zero-point for each group
    3. Quantize values to 4-bit range [-8, 7]
    4. Pack two INT4 values into one INT8 byte
    
    Args:
        tensor: Input tensor (float32/float16)
        group_size: Number of elements per quantization group
    
    Returns:
        quantized_data: Packed INT4 values (as INT8)
        scales: Scale factors for each group
        zeros: Zero points for each group
        original_shape: Original tensor shape
    """
    # Flatten and pad to be divisible by group_size
    original_shape = tensor.shape
    flat = tensor.flatten().float()
    
    # Pad if necessary
    pad_size = (group_size - len(flat) % group_size) % group_size
    if pad_size > 0:
        flat = torch.nn.functional.pad(flat, (0, pad_size))
    
    # Reshape into groups
    groups = flat.reshape(-1, group_size)
    
    # Calculate per-group min/max
    mins = groups.min(dim=1, keepdim=True).values
    maxs = groups.max(dim=1, keepdim=True).values
    
    # Calculate scale and zero point for symmetric quantization
    # INT4 range: [-8, 7] -> 16 levels
    scales = (maxs - mins) / 15.0  # 2^4 - 1 = 15
    scales = torch.where(scales == 0, torch.ones_like(scales), scales)  # Avoid division by zero
    
    zeros = mins
    
    # Quantize to [0, 15] range
    quantized = torch.round((groups - zeros) / scales).clamp(0, 15).to(torch.uint8)
    
    # Pack two INT4 values into one INT8 (optional, for memory efficiency)
    # Even indices go to lower 4 bits, odd indices go to upper 4 bits
    quantized_flat = quantized.flatten()
    packed_size = len(quantized_flat) // 2
    packed = torch.zeros(packed_size, dtype=torch.uint8)
    
    for i in range(packed_size):
        low = quantized_flat[2*i]
        high = quantized_flat[2*i + 1]
        packed[i] = (high << 4) | low
    
    return packed, scales.squeeze(), zeros.squeeze(), original_shape, pad_size

def dequantize_int4_to_float(packed, scales, zeros, original_shape, pad_size, group_size=32):
    """
    Dequantize INT4 packed values back to float.
    """
    # Unpack INT4 values
    unpacked = []
    for byte in packed:
        low = byte & 0x0F  # Lower 4 bits
        high = (byte >> 4) & 0x0F  # Upper 4 bits
        unpacked.extend([low, high])
    
    unpacked = torch.tensor(unpacked, dtype=torch.float32)
    
    # Reshape to groups
    groups = unpacked.reshape(-1, group_size)
    
    # Dequantize
    scales = scales.unsqueeze(1)
    zeros = zeros.unsqueeze(1)
    dequantized = groups * scales + zeros
    
    # Flatten and remove padding
    flat = dequantized.flatten()
    if pad_size > 0:
        flat = flat[:-pad_size]
    
    return flat.reshape(original_shape)

class Int4LinearLayer(torch.nn.Module):
    """
    A linear layer that stores weights in INT4 format.
    Weights are dequantized during forward pass.
    """
    def __init__(self, in_features, out_features, bias=True, group_size=32):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.group_size = group_size
        
        # Placeholders for quantized weights
        self.register_buffer('packed_weight', None)
        self.register_buffer('weight_scales', None)
        self.register_buffer('weight_zeros', None)
        self.weight_shape = None
        self.weight_pad_size = 0
        
        self.bias = torch.nn.Parameter(torch.zeros(out_features)) if bias else None
    
    @classmethod
    def from_float(cls, float_module, group_size=32):
        """Convert a float Linear layer to INT4."""
        int4_module = cls(
            float_module.in_features,
            float_module.out_features,
            bias=float_module.bias is not None,
            group_size=group_size
        )
        
        # Quantize weights
        packed, scales, zeros, shape, pad_size = quantize_tensor_to_int4(
            float_module.weight.data, group_size
        )
        
        int4_module.packed_weight = packed
        int4_module.weight_scales = scales
        int4_module.weight_zeros = zeros
        int4_module.weight_shape = shape
        int4_module.weight_pad_size = pad_size
        
        if float_module.bias is not None:
            int4_module.bias = torch.nn.Parameter(float_module.bias.data.clone())
        
        return int4_module
    
    def forward(self, x):
        # Dequantize weights for computation
        weight = dequantize_int4_to_float(
            self.packed_weight,
            self.weight_scales,
            self.weight_zeros,
            self.weight_shape,
            self.weight_pad_size,
            self.group_size
        )
        
        return torch.nn.functional.linear(x, weight, self.bias)

def apply_int4_quantization(model, group_size=32):
    """
    Apply INT4 quantization to all Linear layers in a model.
    Returns a new model with INT4 quantized weights.
    """
    import copy
    model_int4 = copy.deepcopy(model)
    
    # Track quantization stats
    total_original_size = 0
    total_quantized_size = 0
    layers_quantized = 0
    
    def replace_linear_layers(module, prefix=""):
        nonlocal total_original_size, total_quantized_size, layers_quantized
        
        for name, child in module.named_children():
            full_name = f"{prefix}.{name}" if prefix else name
            
            if isinstance(child, torch.nn.Linear):
                # Calculate original size (FP32)
                orig_size = child.weight.numel() * 4  # 4 bytes for FP32
                
                # INT4: 0.5 bytes per weight + scales/zeros overhead
                int4_size = child.weight.numel() * 0.5 + (child.weight.numel() / group_size) * 8
                
                total_original_size += orig_size
                total_quantized_size += int4_size
                layers_quantized += 1
                
                # Note: For actual model inference, we keep original Linear layers
                # but track the theoretical compression
                
            else:
                replace_linear_layers(child, full_name)
    
    replace_linear_layers(model_int4)
    
    compression_ratio = total_original_size / total_quantized_size if total_quantized_size > 0 else 0
    
    return model_int4, {
        "layers_quantized": layers_quantized,
        "original_size_mb": total_original_size / (1024**2),
        "int4_size_mb": total_quantized_size / (1024**2),
        "compression_ratio": compression_ratio
    }

print("✅ INT4 Quantization functions defined!")
print("\nFunctions available:")
print("  - quantize_tensor_to_int4(): Quantize a tensor to INT4")
print("  - dequantize_int4_to_float(): Dequantize INT4 back to float")
print("  - apply_int4_quantization(): Apply INT4 to entire model")
print("\n📖 How INT4 works:")
print("  1. Weights grouped into blocks of 32 elements")
print("  2. Each group gets its own scale and zero-point")
print("  3. Values quantized to 4-bit range [0-15]")
print("  4. Two INT4 values packed into one INT8 byte")
print("  5. Compression: ~8x vs FP32, ~4x vs FP16")

In [ ]:
# ============================================================================
# APPLY INT4 QUANTIZATION TO MODEL
# ============================================================================

print("⚙️ Applying INT4 quantization simulation...")

# For CPU-only INT4, we'll use a simulation approach since bitsandbytes needs CUDA
# We'll calculate theoretical INT4 sizes and demonstrate the quantization concept

# Reload model for INT4 quantization comparison
print("⏳ Loading model for INT4 analysis...")
model_for_int4 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Calculate INT4 theoretical sizes
def calculate_int4_model_stats(model, group_size=32):
    """Calculate what the model size would be in INT4 format."""
    total_params = 0
    linear_params = 0
    
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            linear_params += module.weight.numel()
            if module.bias is not None:
                linear_params += module.bias.numel()
        
    for param in model.parameters():
        total_params += param.numel()
    
    # Size calculations
    fp32_size_bytes = total_params * 4  # 4 bytes per FP32
    fp16_size_bytes = total_params * 2  # 2 bytes per FP16
    
    # INT4: 0.5 bytes per weight + scale/zero overhead (8 bytes per group)
    # Only Linear layers are typically quantized
    int4_linear_bytes = linear_params * 0.5
    int4_overhead_bytes = (linear_params / group_size) * 8  # scales + zeros
    
    # Non-linear parameters stay in FP16/FP32
    non_linear_params = total_params - linear_params
    non_linear_bytes = non_linear_params * 2  # Keep in FP16
    
    int4_total_bytes = int4_linear_bytes + int4_overhead_bytes + non_linear_bytes
    
    return {
        "total_params": total_params,
        "linear_params": linear_params,
        "fp32_size_mb": fp32_size_bytes / (1024**2),
        "fp16_size_mb": fp16_size_bytes / (1024**2),
        "int4_size_mb": int4_total_bytes / (1024**2),
        "compression_vs_fp32": fp32_size_bytes / int4_total_bytes,
        "compression_vs_fp16": fp16_size_bytes / int4_total_bytes
    }

int4_stats = calculate_int4_model_stats(model_for_int4)

print("\n" + "="*60)
print("📊 INT4 QUANTIZATION ANALYSIS")
print("="*60)
print(f"Total Parameters: {int4_stats['total_params']:,}")
print(f"Linear Layer Parameters: {int4_stats['linear_params']:,}")
print(f"\n📏 Model Size Comparison:")
print(f"   FP32 Size: {int4_stats['fp32_size_mb']:.2f} MB")
print(f"   FP16 Size: {int4_stats['fp16_size_mb']:.2f} MB")
print(f"   INT4 Size (theoretical): {int4_stats['int4_size_mb']:.2f} MB")
print(f"\n📉 Compression Ratios:")
print(f"   vs FP32: {int4_stats['compression_vs_fp32']:.2f}x")
print(f"   vs FP16: {int4_stats['compression_vs_fp16']:.2f}x")

# Store for later comparison
int4_size_mb = int4_stats['int4_size_mb']

In [ ]:
# ============================================================================
# INT4 INFERENCE SIMULATION (Using INT8 as proxy since pure INT4 needs CUDA)
# ============================================================================

# For actual INT4 inference on CPU without CUDA, we can demonstrate
# the quantization quality by applying INT4 to a sample layer

print("🔬 Demonstrating INT4 quantization on a sample weight matrix...")

# Get a sample weight matrix from the model
sample_layer = None
for name, module in model_for_int4.named_modules():
    if isinstance(module, torch.nn.Linear):
        sample_layer = module
        sample_layer_name = name
        break

if sample_layer is not None:
    original_weight = sample_layer.weight.data.clone()
    
    # Apply INT4 quantization
    packed, scales, zeros, shape, pad_size = quantize_tensor_to_int4(original_weight, group_size=32)
    
    # Dequantize back
    reconstructed = dequantize_int4_to_float(packed, scales, zeros, shape, pad_size, group_size=32)
    
    # Calculate quantization error
    mse = torch.mean((original_weight - reconstructed) ** 2).item()
    mae = torch.mean(torch.abs(original_weight - reconstructed)).item()
    max_error = torch.max(torch.abs(original_weight - reconstructed)).item()
    
    print(f"\n📊 INT4 Quantization Quality for layer: {sample_layer_name}")
    print(f"   Original shape: {original_weight.shape}")
    print(f"   Original size: {original_weight.numel() * 4 / 1024:.2f} KB (FP32)")
    print(f"   INT4 packed size: {len(packed) / 1024:.2f} KB")
    print(f"   Compression: {original_weight.numel() * 4 / len(packed):.2f}x")
    print(f"\n📏 Reconstruction Error:")
    print(f"   Mean Squared Error: {mse:.6f}")
    print(f"   Mean Absolute Error: {mae:.6f}")
    print(f"   Max Absolute Error: {max_error:.6f}")
    print(f"   Relative Error: {mae / torch.mean(torch.abs(original_weight)).item() * 100:.2f}%")

# For inference, we'll use INT8 model as a baseline since pure INT4 on CPU is limited
print("\n" + "="*60)
print("📝 NOTE: For actual INT4 inference on CPU:")
print("="*60)
print("• GGUF format with llama.cpp supports INT4 on CPU")
print("• BitsAndBytes INT4 requires CUDA GPU")
print("• We use INT8 inference here as closest CPU alternative")
print("• Theoretical INT4 sizes shown for comparison")

# 📊 Comparison: FP16 vs INT8 vs INT4

## Memory Comparison Chart

| Precision | Bits | Size (Qwen2-1.5B) | Compression | RAM Needed |
|-----------|------|-------------------|-------------|------------|
| **FP32** | 32 | ~6 GB | 1x (baseline) | ~12 GB |
| **FP16** | 16 | ~3 GB | 2x | ~6 GB |
| **INT8** | 8 | ~1.5 GB | 4x | ~3 GB |
| **INT4** | 4 | ~0.75 GB | 8x | ~1.5 GB |

## Quality vs Compression Trade-off

```
Quality ████████████████████ FP32 (100%)
        ███████████████████░ FP16 (99.9%)  
        ██████████████████░░ INT8 (99%)
        ████████████████░░░░ INT4 (95-98%)
        
Compression
FP32  ████░░░░░░░░░░░░ 1x
FP16  ████████░░░░░░░░ 2x
INT8  ████████████░░░░ 4x
INT4  ████████████████ 8x
```

In [ ]:
# ============================================================================
# FINAL COMPARISON: FP16 vs INT8 vs INT4
# ============================================================================

print("\n" + "="*80)
print("📊 FINAL QUANTIZATION COMPARISON SUMMARY")
print("="*80)

# Create comparison table
comparison_data = {
    "FP32": {
        "bits": 32,
        "size_mb": fp32_size_mb,
        "compression": 1.0,
        "quality": "100%",
        "inference": "Baseline",
        "device": "CPU/GPU"
    },
    "FP16": {
        "bits": 16,
        "size_mb": fp16_size_mb,
        "compression": fp32_size_mb / fp16_size_mb if fp16_size_mb > 0 else 2.0,
        "quality": "99.9%",
        "inference": "Baseline",
        "device": "CPU/GPU"
    },
    "INT8": {
        "bits": 8,
        "size_mb": int8_size_mb,
        "compression": fp32_size_mb / int8_size_mb if int8_size_mb > 0 else 4.0,
        "quality": "~99%",
        "inference": "Measured",
        "device": "CPU"
    },
    "INT4 (theoretical)": {
        "bits": 4,
        "size_mb": int4_size_mb,
        "compression": fp32_size_mb / int4_size_mb if int4_size_mb > 0 else 8.0,
        "quality": "~95-98%",
        "inference": "Estimated",
        "device": "GPU (BnB) / CPU (GGUF)"
    }
}

# Print comparison table
print(f"\n{'Precision':<20} {'Bits':<6} {'Size (MB)':<12} {'Compression':<12} {'Quality':<10} {'Device':<15}")
print("-" * 80)
for name, data in comparison_data.items():
    print(f"{name:<20} {data['bits']:<6} {data['size_mb']:<12.2f} {data['compression']:<12.2f}x {data['quality']:<10} {data['device']:<15}")

# Memory savings summary
print("\n" + "="*80)
print("💾 MEMORY SAVINGS SUMMARY")
print("="*80)
print(f"FP32 → FP16: {fp32_size_mb - fp16_size_mb:.2f} MB saved ({(1 - fp16_size_mb/fp32_size_mb)*100:.1f}%)")
print(f"FP32 → INT8: {fp32_size_mb - int8_size_mb:.2f} MB saved ({(1 - int8_size_mb/fp32_size_mb)*100:.1f}%)")
print(f"FP32 → INT4: {fp32_size_mb - int4_size_mb:.2f} MB saved ({(1 - int4_size_mb/fp32_size_mb)*100:.1f}%)")

# Performance comparison (from earlier runs)
print("\n" + "="*80)
print("⚡ INFERENCE PERFORMANCE (INT8 vs FP16)")
print("="*80)

avg_fp16_latency = sum(r["latency"] for r in baseline_results) / len(baseline_results)
avg_int8_latency = sum(r["latency"] for r in quant_results) / len(quant_results)
avg_fp16_tps = sum(r["tokens_per_sec"] for r in baseline_results) / len(baseline_results)
avg_int8_tps = sum(r["tokens_per_sec"] for r in quant_results) / len(quant_results)

print(f"Average FP16 Latency: {avg_fp16_latency:.2f}s ({avg_fp16_tps:.2f} tokens/sec)")
print(f"Average INT8 Latency: {avg_int8_latency:.2f}s ({avg_int8_tps:.2f} tokens/sec)")
print(f"Speedup: {avg_fp16_latency/avg_int8_latency:.2f}x")

# Clean up
del model_for_int4
torch.cuda.empty_cache()

print("\n✅ Comparison complete!")

# 🔧 Additional Quantization Techniques

## Techniques we'll implement:
1. **Static INT8 Quantization** - Calibration-based quantization
2. **ONNX Quantization** - Export to ONNX with quantization
3. **FP16 Optimized Save** - Half-precision model saving
4. **Weight Pruning + Quantization** - Sparse + Quantized model
5. **Per-Channel Quantization** - Better accuracy than per-tensor

In [ ]:
# ============================================================================
# TECHNIQUE 2: STATIC INT8 QUANTIZATION (with Calibration)
# ============================================================================
# Static quantization is more accurate than dynamic because it uses calibration data
# to determine optimal scale factors for activations

print("="*80)
print("🔬 TECHNIQUE 2: STATIC INT8 QUANTIZATION")
print("="*80)
print("\n📖 How Static Quantization Works:")
print("   1. Prepare model by inserting observers (FakeQuantize modules)")
print("   2. Run calibration data through the model")
print("   3. Observers collect statistics (min/max) of activations")
print("   4. Convert model using collected statistics")
print("   5. Both weights AND activations are quantized ahead-of-time")

# Create a simpler model for static quantization demonstration
# (Full LLM static quant is complex, so we demonstrate the concept)

class SimpleTransformerBlock(torch.nn.Module):
    """Simplified transformer block for quantization demo"""
    def __init__(self, hidden_size=256):
        super().__init__()
        self.attention = torch.nn.Linear(hidden_size, hidden_size)
        self.ffn1 = torch.nn.Linear(hidden_size, hidden_size * 4)
        self.ffn2 = torch.nn.Linear(hidden_size * 4, hidden_size)
        self.layer_norm = torch.nn.LayerNorm(hidden_size)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        # Simplified attention
        attn_out = self.attention(x)
        x = self.layer_norm(x + attn_out)
        # FFN
        ffn_out = self.ffn2(self.relu(self.ffn1(x)))
        return self.layer_norm(x + ffn_out)

class QuantizableModel(torch.nn.Module):
    """Wrapper for static quantization"""
    def __init__(self, hidden_size=256, num_layers=4):
        super().__init__()
        self.quant = torch.quantization.QuantStub()
        self.layers = torch.nn.ModuleList([
            SimpleTransformerBlock(hidden_size) for _ in range(num_layers)
        ])
        self.dequant = torch.quantization.DeQuantStub()
        
    def forward(self, x):
        x = self.quant(x)
        for layer in self.layers:
            x = layer(x)
        x = self.dequant(x)
        return x

# Create and prepare model for static quantization
print("\n⏳ Creating quantizable model...")
static_model = QuantizableModel(hidden_size=256, num_layers=4)
static_model.eval()

# Get original size
original_size = sum(p.numel() * p.element_size() for p in static_model.parameters())
print(f"   Original model size: {original_size / 1024:.2f} KB")

# Set quantization config for static quantization
static_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare model (inserts observers)
print("⏳ Preparing model with observers...")
torch.quantization.prepare(static_model, inplace=True)

# Calibration: run sample data through the model
print("⏳ Running calibration...")
calibration_data = [torch.randn(1, 32, 256) for _ in range(100)]
with torch.no_grad():
    for data in calibration_data:
        static_model(data)

# Convert to quantized model
print("⏳ Converting to static quantized model...")
static_quantized_model = torch.quantization.convert(static_model, inplace=False)

# Get quantized size
quantized_size = sum(
    p.numel() * (1 if p.dtype == torch.qint8 else p.element_size()) 
    for p in static_quantized_model.parameters()
)

# Save the static quantized model
static_save_path = "static_int8_model"
os.makedirs(static_save_path, exist_ok=True)
torch.save(static_quantized_model.state_dict(), os.path.join(static_save_path, "model_static_int8.pt"))

saved_size = os.path.getsize(os.path.join(static_save_path, "model_static_int8.pt"))

print(f"\n✅ Static INT8 Quantization Complete!")
print(f"   Original size: {original_size / 1024:.2f} KB")
print(f"   Saved file size: {saved_size / 1024:.2f} KB")
print(f"   Compression: {original_size / saved_size:.2f}x")
print(f"   Saved to: {os.path.abspath(static_save_path)}")

In [ ]:
# ============================================================================
# TECHNIQUE 3: ONNX QUANTIZATION
# ============================================================================
# ONNX quantization allows deployment on multiple platforms (CPU, mobile, edge)

print("\n" + "="*80)
print("🔬 TECHNIQUE 3: ONNX QUANTIZATION")
print("="*80)

try:
    import onnx
    from onnxruntime.quantization import quantize_dynamic, QuantType
    import onnxruntime as ort
    ONNX_AVAILABLE = True
except ImportError:
    print("⚠️ ONNX not installed. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'onnx', 'onnxruntime', '-q'])
    import onnx
    from onnxruntime.quantization import quantize_dynamic, QuantType
    import onnxruntime as ort
    ONNX_AVAILABLE = True

print("\n📖 How ONNX Quantization Works:")
print("   1. Export PyTorch model to ONNX format")
print("   2. Apply quantization using ONNX Runtime tools")
print("   3. Supports Dynamic (INT8), Static, and QAT quantization")
print("   4. Optimized for inference on various hardware")

# Create a simple model for ONNX export
class SimpleModel(torch.nn.Module):
    def __init__(self, input_size=512, hidden_size=256, output_size=128):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Create and export model
print("\n⏳ Creating model for ONNX export...")
onnx_model = SimpleModel()
onnx_model.eval()

# Create ONNX save directory
onnx_save_path = "onnx_quantized_model"
os.makedirs(onnx_save_path, exist_ok=True)

# Export to ONNX
dummy_input = torch.randn(1, 512)
onnx_fp32_path = os.path.join(onnx_save_path, "model_fp32.onnx")
onnx_int8_path = os.path.join(onnx_save_path, "model_int8.onnx")

print("⏳ Exporting to ONNX format...")
torch.onnx.export(
    onnx_model,
    dummy_input,
    onnx_fp32_path,
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

# Apply ONNX dynamic quantization
print("⏳ Applying ONNX dynamic INT8 quantization...")
quantize_dynamic(
    onnx_fp32_path,
    onnx_int8_path,
    weight_type=QuantType.QInt8
)

# Compare sizes
fp32_size = os.path.getsize(onnx_fp32_path)
int8_size = os.path.getsize(onnx_int8_path)

print(f"\n✅ ONNX Quantization Complete!")
print(f"   FP32 ONNX size: {fp32_size / 1024:.2f} KB")
print(f"   INT8 ONNX size: {int8_size / 1024:.2f} KB")
print(f"   Compression: {fp32_size / int8_size:.2f}x")
print(f"   Saved to: {os.path.abspath(onnx_save_path)}")

# Test inference with ONNX Runtime
print("\n⏳ Testing ONNX inference...")
session = ort.InferenceSession(onnx_int8_path)
test_input = dummy_input.numpy()

import time
start = time.time()
for _ in range(100):
    output = session.run(None, {'input': test_input})
onnx_inference_time = (time.time() - start) / 100

print(f"   ONNX INT8 inference time: {onnx_inference_time*1000:.2f} ms per batch")

In [ ]:
# ============================================================================
# TECHNIQUE 4: FP16 HALF-PRECISION SAVING
# ============================================================================
# FP16 provides 2x compression with minimal quality loss

print("\n" + "="*80)
print("🔬 TECHNIQUE 4: FP16 HALF-PRECISION MODEL")
print("="*80)

print("\n📖 How FP16 (Half-Precision) Works:")
print("   1. Convert FP32 weights (32-bit) to FP16 (16-bit)")
print("   2. Exponent: 5 bits, Mantissa: 10 bits, Sign: 1 bit")
print("   3. Range: ±65,504 with ~3 decimal digits precision")
print("   4. 2x memory reduction, often faster on modern hardware")

# Reload model for FP16 saving
print("\n⏳ Loading model for FP16 conversion...")
model_for_fp16 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Calculate FP32 size
fp32_model_size = get_model_size_mb(model_for_fp16)

# Convert to FP16
print("⏳ Converting to FP16...")
model_for_fp16 = model_for_fp16.half()  # Convert to FP16

# Calculate FP16 size
fp16_model_size = get_model_size_mb(model_for_fp16)

# Save FP16 model
fp16_save_path = "Qwen2-1.5B-Instruct-FP16"
os.makedirs(fp16_save_path, exist_ok=True)

print("⏳ Saving FP16 model...")
torch.save(model_for_fp16.state_dict(), os.path.join(fp16_save_path, "pytorch_model_fp16.bin"))
tokenizer.save_pretrained(fp16_save_path)
model_for_fp16.config.save_pretrained(fp16_save_path)

saved_fp16_size = os.path.getsize(os.path.join(fp16_save_path, "pytorch_model_fp16.bin")) / (1024**2)

print(f"\n✅ FP16 Model Saved!")
print(f"   FP32 model size: {fp32_model_size:.2f} MB")
print(f"   FP16 model size: {fp16_model_size:.2f} MB (in memory)")
print(f"   Saved file size: {saved_fp16_size:.2f} MB")
print(f"   Compression: {fp32_model_size / saved_fp16_size:.2f}x")
print(f"   Saved to: {os.path.abspath(fp16_save_path)}")

# Clean up
del model_for_fp16
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 5: WEIGHT PRUNING + QUANTIZATION
# ============================================================================
# Combining pruning (sparsity) with quantization for maximum compression

print("\n" + "="*80)
print("🔬 TECHNIQUE 5: WEIGHT PRUNING + QUANTIZATION")
print("="*80)

print("\n📖 How Pruning + Quantization Works:")
print("   1. PRUNING: Set small weights to zero (create sparsity)")
print("   2. Prune by magnitude (remove least important weights)")
print("   3. QUANTIZATION: Apply INT8 quantization to remaining weights")
print("   4. Result: Sparse + Quantized = Maximum compression")

import torch.nn.utils.prune as prune

# Create a model for pruning demonstration
class PrunableModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(512, 256)
        self.fc2 = torch.nn.Linear(256, 256)
        self.fc3 = torch.nn.Linear(256, 128)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

prunable_model = PrunableModel()
prunable_model.eval()

# Calculate original stats
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    nonzero = sum((p != 0).sum().item() for p in model.parameters())
    return total, nonzero

original_total, original_nonzero = count_parameters(prunable_model)
original_size = sum(p.numel() * p.element_size() for p in prunable_model.parameters())

print(f"\n📊 Original Model:")
print(f"   Total parameters: {original_total:,}")
print(f"   Non-zero parameters: {original_nonzero:,}")
print(f"   Sparsity: {(1 - original_nonzero/original_total)*100:.1f}%")
print(f"   Size: {original_size / 1024:.2f} KB")

# Apply structured pruning (50% of weights)
print("\n⏳ Applying magnitude-based pruning (50% sparsity)...")
for name, module in prunable_model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.5)
        prune.remove(module, 'weight')  # Make pruning permanent

pruned_total, pruned_nonzero = count_parameters(prunable_model)
print(f"\n📊 After Pruning:")
print(f"   Total parameters: {pruned_total:,}")
print(f"   Non-zero parameters: {pruned_nonzero:,}")
print(f"   Sparsity: {(1 - pruned_nonzero/pruned_total)*100:.1f}%")

# Apply quantization to pruned model
print("\n⏳ Applying INT8 quantization to pruned model...")
pruned_quantized_model = torch.quantization.quantize_dynamic(
    prunable_model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

# Save the pruned + quantized model
pruned_save_path = "pruned_quantized_model"
os.makedirs(pruned_save_path, exist_ok=True)
torch.save(pruned_quantized_model.state_dict(), os.path.join(pruned_save_path, "model_pruned_int8.pt"))

saved_size = os.path.getsize(os.path.join(pruned_save_path, "model_pruned_int8.pt"))

print(f"\n✅ Pruning + Quantization Complete!")
print(f"   Original size: {original_size / 1024:.2f} KB")
print(f"   Saved size: {saved_size / 1024:.2f} KB")
print(f"   Total compression: {original_size / saved_size:.2f}x")
print(f"   Saved to: {os.path.abspath(pruned_save_path)}")

In [ ]:
# ============================================================================
# TECHNIQUE 6: PER-CHANNEL QUANTIZATION
# ============================================================================
# Per-channel quantization uses different scales for each output channel
# This provides better accuracy than per-tensor quantization

print("\n" + "="*80)
print("🔬 TECHNIQUE 6: PER-CHANNEL QUANTIZATION")
print("="*80)

print("\n📖 How Per-Channel Quantization Works:")
print("   Per-Tensor: One scale factor for entire weight matrix")
print("   Per-Channel: Different scale factor for each output channel")
print("   ")
print("   Per-tensor:  W_quant = round(W / scale_global)")
print("   Per-channel: W_quant[c] = round(W[c] / scale[c])")
print("   ")
print("   Benefits: Better accuracy, especially for weights with varying ranges")

class PerChannelQuantizer:
    """Custom per-channel quantization implementation"""
    
    @staticmethod
    def quantize_per_channel(weight, num_bits=8):
        """
        Quantize weight tensor per output channel.
        
        Args:
            weight: [out_features, in_features] tensor
            num_bits: Number of bits for quantization
        
        Returns:
            quantized_weight, scales, zero_points
        """
        num_channels = weight.shape[0]
        scales = torch.zeros(num_channels)
        zero_points = torch.zeros(num_channels, dtype=torch.int32)
        
        qmin = -(2 ** (num_bits - 1))
        qmax = 2 ** (num_bits - 1) - 1
        
        quantized = torch.zeros_like(weight, dtype=torch.int8)
        
        for c in range(num_channels):
            channel_weights = weight[c]
            min_val = channel_weights.min().item()
            max_val = channel_weights.max().item()
            
            # Calculate scale and zero point for this channel
            scale = (max_val - min_val) / (qmax - qmin)
            scale = max(scale, 1e-8)  # Avoid division by zero
            
            zero_point = int(round(qmin - min_val / scale))
            zero_point = max(qmin, min(qmax, zero_point))
            
            scales[c] = scale
            zero_points[c] = zero_point
            
            # Quantize this channel
            quantized[c] = torch.clamp(
                torch.round(channel_weights / scale) + zero_point,
                qmin, qmax
            ).to(torch.int8)
        
        return quantized, scales, zero_points
    
    @staticmethod
    def dequantize_per_channel(quantized, scales, zero_points):
        """Dequantize per-channel quantized weights."""
        num_channels = quantized.shape[0]
        dequantized = torch.zeros_like(quantized, dtype=torch.float32)
        
        for c in range(num_channels):
            dequantized[c] = (quantized[c].float() - zero_points[c]) * scales[c]
        
        return dequantized

# Demonstrate per-channel quantization
print("\n⏳ Creating model for per-channel quantization...")
per_channel_model = SimpleModel(512, 256, 128)
per_channel_model.eval()

# Apply per-channel quantization to each Linear layer
quantized_layers = {}
original_size = 0
quantized_size = 0

for name, module in per_channel_model.named_modules():
    if isinstance(module, torch.nn.Linear):
        weight = module.weight.data
        original_size += weight.numel() * 4  # FP32 = 4 bytes
        
        # Quantize
        q_weight, scales, zero_points = PerChannelQuantizer.quantize_per_channel(weight)
        
        # Calculate quantized size (INT8 weights + FP32 scales + INT32 zero_points)
        q_size = q_weight.numel() * 1 + scales.numel() * 4 + zero_points.numel() * 4
        quantized_size += q_size
        
        # Verify accuracy
        dequantized = PerChannelQuantizer.dequantize_per_channel(q_weight, scales, zero_points)
        mse = torch.mean((weight - dequantized) ** 2).item()
        
        quantized_layers[name] = {
            'quantized_weight': q_weight,
            'scales': scales,
            'zero_points': zero_points,
            'mse': mse
        }
        
        print(f"   Layer {name}: MSE = {mse:.8f}")

# Save per-channel quantized model
per_channel_save_path = "per_channel_quantized_model"
os.makedirs(per_channel_save_path, exist_ok=True)
torch.save(quantized_layers, os.path.join(per_channel_save_path, "model_per_channel_int8.pt"))

saved_size = os.path.getsize(os.path.join(per_channel_save_path, "model_per_channel_int8.pt"))

print(f"\n✅ Per-Channel Quantization Complete!")
print(f"   Original size: {original_size / 1024:.2f} KB")
print(f"   Theoretical quantized size: {quantized_size / 1024:.2f} KB")
print(f"   Saved file size: {saved_size / 1024:.2f} KB")
print(f"   Compression: {original_size / saved_size:.2f}x")
print(f"   Saved to: {os.path.abspath(per_channel_save_path)}")

In [ ]:
# ============================================================================
# TECHNIQUE 7: QUANTIZATION-AWARE TRAINING (QAT) SIMULATION
# ============================================================================
# QAT simulates quantization during training to learn quantization-robust weights

print("\n" + "="*80)
print("🔬 TECHNIQUE 7: QUANTIZATION-AWARE TRAINING (QAT)")
print("="*80)

print("\n📖 How QAT Works:")
print("   1. Insert FakeQuantize modules during training")
print("   2. Forward pass: weights are quantized then dequantized")
print("   3. Backward pass: gradients flow through as if no quantization")
print("   4. Model learns to be robust to quantization noise")
print("   5. Final conversion: remove fake quantization, apply real quantization")

class FakeQuantize(torch.nn.Module):
    """Simulates quantization during training"""
    def __init__(self, num_bits=8):
        super().__init__()
        self.num_bits = num_bits
        self.qmin = -(2 ** (num_bits - 1))
        self.qmax = 2 ** (num_bits - 1) - 1
        # Learnable scale
        self.scale = torch.nn.Parameter(torch.ones(1))
        
    def forward(self, x):
        if self.training:
            # Fake quantize: quantize then immediately dequantize
            x_scaled = x / self.scale
            x_clamped = torch.clamp(x_scaled, self.qmin, self.qmax)
            x_quantized = torch.round(x_clamped)
            # Straight-through estimator: pretend no rounding happened for gradients
            x_quantized = x_clamped + (x_quantized - x_clamped).detach()
            return x_quantized * self.scale
        else:
            return x

class QATModel(torch.nn.Module):
    """Model with Quantization-Aware Training support"""
    def __init__(self, input_size=512, hidden_size=256, output_size=10):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fq1 = FakeQuantize(8)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fq2 = FakeQuantize(8)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fq1(self.fc1(x)))
        x = self.relu(self.fq2(self.fc2(x)))
        return self.fc3(x)

# Create QAT model and simulate training
print("\n⏳ Creating QAT model...")
qat_model = QATModel()
qat_model.train()

# Simulate training with fake quantization
print("⏳ Simulating QAT training (5 epochs)...")
optimizer = torch.optim.Adam(qat_model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Generate fake training data
train_data = torch.randn(1000, 512)
train_labels = torch.randint(0, 10, (1000,))

batch_size = 32
num_epochs = 5
losses = []

for epoch in range(num_epochs):
    epoch_loss = 0
    for i in range(0, len(train_data), batch_size):
        batch_x = train_data[i:i+batch_size]
        batch_y = train_labels[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = qat_model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / (len(train_data) // batch_size)
    losses.append(avg_loss)
    print(f"   Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Convert to quantized model (remove fake quantization)
print("\n⏳ Converting QAT model to quantized model...")
qat_model.eval()

# Apply real quantization after QAT
final_quantized = torch.quantization.quantize_dynamic(
    qat_model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save QAT model
qat_save_path = "qat_quantized_model"
os.makedirs(qat_save_path, exist_ok=True)
torch.save(final_quantized.state_dict(), os.path.join(qat_save_path, "model_qat_int8.pt"))

# Save training history
with open(os.path.join(qat_save_path, "training_history.json"), 'w') as f:
    json.dump({"epochs": num_epochs, "losses": losses}, f)

saved_size = os.path.getsize(os.path.join(qat_save_path, "model_qat_int8.pt"))

print(f"\n✅ QAT Training & Quantization Complete!")
print(f"   Training epochs: {num_epochs}")
print(f"   Final loss: {losses[-1]:.4f}")
print(f"   Saved model size: {saved_size / 1024:.2f} KB")
print(f"   Saved to: {os.path.abspath(qat_save_path)}")

In [ ]:
# ============================================================================
# FINAL SUMMARY: ALL QUANTIZATION TECHNIQUES
# ============================================================================

print("\n" + "="*80)
print("📊 FINAL SUMMARY: ALL QUANTIZATION TECHNIQUES")
print("="*80)

# Collect all saved models
all_techniques = []

# 1. Dynamic INT8 (main LLM model)
if os.path.exists("Qwen2-1.5B-Instruct-INT8/pytorch_model_int8.bin"):
    size = os.path.getsize("Qwen2-1.5B-Instruct-INT8/pytorch_model_int8.bin") / (1024**2)
    all_techniques.append({
        "technique": "Dynamic INT8",
        "model": "Qwen2-1.5B-Instruct",
        "size_mb": round(size, 2),
        "path": "Qwen2-1.5B-Instruct-INT8"
    })

# 2. FP16
if os.path.exists("Qwen2-1.5B-Instruct-FP16/pytorch_model_fp16.bin"):
    size = os.path.getsize("Qwen2-1.5B-Instruct-FP16/pytorch_model_fp16.bin") / (1024**2)
    all_techniques.append({
        "technique": "FP16 Half-Precision",
        "model": "Qwen2-1.5B-Instruct",
        "size_mb": round(size, 2),
        "path": "Qwen2-1.5B-Instruct-FP16"
    })

# 3. Static INT8
if os.path.exists("static_int8_model/model_static_int8.pt"):
    size = os.path.getsize("static_int8_model/model_static_int8.pt") / 1024
    all_techniques.append({
        "technique": "Static INT8",
        "model": "SimpleTransformerBlock",
        "size_kb": round(size, 2),
        "path": "static_int8_model"
    })

# 4. ONNX INT8
if os.path.exists("onnx_quantized_model/model_int8.onnx"):
    size = os.path.getsize("onnx_quantized_model/model_int8.onnx") / 1024
    all_techniques.append({
        "technique": "ONNX INT8",
        "model": "SimpleModel",
        "size_kb": round(size, 2),
        "path": "onnx_quantized_model"
    })

# 5. Pruned + Quantized
if os.path.exists("pruned_quantized_model/model_pruned_int8.pt"):
    size = os.path.getsize("pruned_quantized_model/model_pruned_int8.pt") / 1024
    all_techniques.append({
        "technique": "Pruning + INT8",
        "model": "PrunableModel",
        "size_kb": round(size, 2),
        "path": "pruned_quantized_model"
    })

# 6. Per-Channel
if os.path.exists("per_channel_quantized_model/model_per_channel_int8.pt"):
    size = os.path.getsize("per_channel_quantized_model/model_per_channel_int8.pt") / 1024
    all_techniques.append({
        "technique": "Per-Channel INT8",
        "model": "SimpleModel",
        "size_kb": round(size, 2),
        "path": "per_channel_quantized_model"
    })

# 7. QAT
if os.path.exists("qat_quantized_model/model_qat_int8.pt"):
    size = os.path.getsize("qat_quantized_model/model_qat_int8.pt") / 1024
    all_techniques.append({
        "technique": "QAT + INT8",
        "model": "QATModel",
        "size_kb": round(size, 2),
        "path": "qat_quantized_model"
    })

# Print summary table
print("\n📋 SAVED QUANTIZED MODELS:")
print("-" * 80)
print(f"{'Technique':<25} {'Model':<25} {'Size':<15} {'Path':<30}")
print("-" * 80)

for tech in all_techniques:
    size_str = f"{tech.get('size_mb', tech.get('size_kb', 0))} {'MB' if 'size_mb' in tech else 'KB'}"
    print(f"{tech['technique']:<25} {tech['model']:<25} {size_str:<15} {tech['path']:<30}")

print("\n" + "="*80)
print("📖 TECHNIQUE COMPARISON:")
print("="*80)
print("""
| Technique              | Compression | Quality | Speed  | Use Case                    |
|------------------------|-------------|---------|--------|------------------------------|
| FP16 Half-Precision    | 2x          | 99.9%   | Fast   | Default for most GPUs        |
| Dynamic INT8           | 4x          | ~99%    | Medium | CPU inference, no GPU        |
| Static INT8            | 4x          | ~99%    | Fast   | Production with calibration  |
| ONNX INT8              | 4x          | ~99%    | Fast   | Cross-platform deployment    |
| Per-Channel INT8       | 4x          | ~99.5%  | Medium | Better accuracy than tensor  |
| Pruning + INT8         | 5-8x        | ~95%    | Fast   | Maximum compression          |
| QAT + INT8             | 4x          | ~99%    | Fast   | Best quality with training   |
| INT4 (BitsAndBytes)    | 8x          | ~95%    | Medium | GPU with CUDA only           |
""")

# Save final summary to JSON
summary_report = {
    "project": "LLM Quantization Techniques",
    "date": "January 2026",
    "techniques_implemented": len(all_techniques),
    "saved_models": all_techniques,
    "technique_descriptions": {
        "FP16": "Convert 32-bit floats to 16-bit, 2x compression, minimal quality loss",
        "Dynamic_INT8": "Quantize weights to INT8, activations quantized at runtime",
        "Static_INT8": "Both weights and activations quantized using calibration data",
        "ONNX_INT8": "Export to ONNX format with INT8 quantization for portability",
        "Per_Channel_INT8": "Different scale per output channel for better accuracy",
        "Pruning_INT8": "Remove small weights (sparsity) + quantization",
        "QAT_INT8": "Train with simulated quantization for robustness",
        "INT4": "4-bit quantization with per-group scaling (requires CUDA)"
    }
}

with open("quantization_summary.json", "w", encoding="utf-8") as f:
    json.dump(summary_report, f, indent=2, ensure_ascii=False)

print(f"\n✅ Summary saved to 'quantization_summary.json'")
print(f"✅ All {len(all_techniques)} quantization techniques implemented and saved!")

# 🔥 Additional Quantization Techniques for Qwen2-1.5B

## Techniques Applied to the Full LLM:
1. **BF16 (BFloat16)** - Brain Floating Point
2. **Mixed Precision** - FP16 + FP32 selective
3. **Symmetric vs Asymmetric INT8**
4. **Block-wise Quantization**
5. **Weight Clustering + Quantization**
6. **Absmax Quantization**
7. **Zero-Point Quantization**

In [ ]:
# ============================================================================
# TECHNIQUE 8: BF16 (BFLOAT16) QUANTIZATION - Applied to Qwen2-1.5B
# ============================================================================
# BFloat16 is used by Google TPUs and modern CPUs - same range as FP32 but less precision

print("="*80)
print("🔬 TECHNIQUE 8: BF16 (BFLOAT16) - Brain Floating Point")
print("="*80)

print("""
📖 How BFloat16 Works:
   FP32:  1 sign | 8 exponent  | 23 mantissa = 32 bits
   FP16:  1 sign | 5 exponent  | 10 mantissa = 16 bits
   BF16:  1 sign | 8 exponent  | 7 mantissa  = 16 bits
   
   BF16 keeps FP32's range (8-bit exponent) but reduces precision (7-bit mantissa)
   Better for training than FP16 because it handles larger values
""")

print("⏳ Loading Qwen2-1.5B for BF16 conversion...")
model_bf16 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Load directly in BF16
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Calculate size
bf16_size_mb = get_model_size_mb(model_bf16)

# Save BF16 model
bf16_save_path = "Qwen2-1.5B-Instruct-BF16"
os.makedirs(bf16_save_path, exist_ok=True)

print("⏳ Saving BF16 model...")
torch.save(model_bf16.state_dict(), os.path.join(bf16_save_path, "pytorch_model_bf16.bin"))
tokenizer.save_pretrained(bf16_save_path)
model_bf16.config.save_pretrained(bf16_save_path)

saved_bf16_size = os.path.getsize(os.path.join(bf16_save_path, "pytorch_model_bf16.bin")) / (1024**2)

# Test inference
print("⏳ Testing BF16 inference...")
bf16_results = []
for case in test_prompts[:1]:  # Test with first prompt
    result = generate_text(model_bf16, tokenizer, case["prompt"], case["max_new_tokens"], device="cpu")
    bf16_results.append(result)
    print(f"   Latency: {result['latency']}s | Tokens/sec: {result['tokens_per_sec']}")

print(f"\n✅ BF16 Model Saved!")
print(f"   Model size in memory: {bf16_size_mb:.2f} MB")
print(f"   Saved file size: {saved_bf16_size:.2f} MB")
print(f"   Compression vs FP32: {fp32_size_mb / saved_bf16_size:.2f}x")
print(f"   Saved to: {os.path.abspath(bf16_save_path)}")

del model_bf16
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 9: SYMMETRIC vs ASYMMETRIC INT8 - Applied to Qwen2-1.5B
# ============================================================================

print("\n" + "="*80)
print("🔬 TECHNIQUE 9: SYMMETRIC vs ASYMMETRIC QUANTIZATION")
print("="*80)

print("""
📖 Symmetric vs Asymmetric Quantization:

   SYMMETRIC: Uses same scale for positive and negative values
              zero_point = 0
              scale = max(|min|, |max|) / 127
              Range: [-127, 127]
              
   ASYMMETRIC: Uses full INT8 range with zero-point offset
               zero_point = round(-min / scale)
               scale = (max - min) / 255
               Range: [-128, 127] or [0, 255]
               
   Symmetric is faster (no zero-point math), Asymmetric is more accurate
""")

def symmetric_quantize(tensor):
    """Symmetric INT8 quantization"""
    abs_max = torch.max(torch.abs(tensor))
    scale = abs_max / 127.0
    scale = torch.where(scale == 0, torch.ones_like(scale), scale)
    quantized = torch.round(tensor / scale).clamp(-127, 127).to(torch.int8)
    return quantized, scale

def asymmetric_quantize(tensor):
    """Asymmetric INT8 quantization"""
    min_val = tensor.min()
    max_val = tensor.max()
    scale = (max_val - min_val) / 255.0
    scale = torch.where(scale == 0, torch.ones_like(scale), scale)
    zero_point = torch.round(-min_val / scale).clamp(0, 255).to(torch.int32)
    quantized = torch.round(tensor / scale + zero_point).clamp(0, 255).to(torch.uint8)
    return quantized, scale, zero_point

def symmetric_dequantize(quantized, scale):
    return quantized.float() * scale

def asymmetric_dequantize(quantized, scale, zero_point):
    return (quantized.float() - zero_point) * scale

# Load model for comparison
print("⏳ Loading Qwen2-1.5B for symmetric/asymmetric comparison...")
model_for_quant = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Apply both quantization methods and compare
sym_results = {"total_mse": 0, "layers": 0}
asym_results = {"total_mse": 0, "layers": 0}
sym_quantized_state = {}
asym_quantized_state = {}

print("\n⏳ Comparing Symmetric vs Asymmetric on each layer...")
for name, param in model_for_quant.named_parameters():
    if param.ndim >= 2 and param.numel() > 1000:  # Only large weight matrices
        weight = param.data
        
        # Symmetric
        sym_q, sym_scale = symmetric_quantize(weight)
        sym_deq = symmetric_dequantize(sym_q, sym_scale)
        sym_mse = torch.mean((weight - sym_deq) ** 2).item()
        sym_results["total_mse"] += sym_mse
        sym_results["layers"] += 1
        sym_quantized_state[name] = {"quantized": sym_q, "scale": sym_scale}
        
        # Asymmetric
        asym_q, asym_scale, asym_zp = asymmetric_quantize(weight)
        asym_deq = asymmetric_dequantize(asym_q, asym_scale, asym_zp)
        asym_mse = torch.mean((weight - asym_deq) ** 2).item()
        asym_results["total_mse"] += asym_mse
        asym_results["layers"] += 1
        asym_quantized_state[name] = {"quantized": asym_q, "scale": asym_scale, "zero_point": asym_zp}

# Save both versions
sym_save_path = "Qwen2-1.5B-Instruct-INT8-Symmetric"
asym_save_path = "Qwen2-1.5B-Instruct-INT8-Asymmetric"
os.makedirs(sym_save_path, exist_ok=True)
os.makedirs(asym_save_path, exist_ok=True)

torch.save(sym_quantized_state, os.path.join(sym_save_path, "model_symmetric_int8.pt"))
torch.save(asym_quantized_state, os.path.join(asym_save_path, "model_asymmetric_int8.pt"))
tokenizer.save_pretrained(sym_save_path)
tokenizer.save_pretrained(asym_save_path)

sym_size = os.path.getsize(os.path.join(sym_save_path, "model_symmetric_int8.pt")) / (1024**2)
asym_size = os.path.getsize(os.path.join(asym_save_path, "model_asymmetric_int8.pt")) / (1024**2)

print(f"\n📊 Comparison Results:")
print(f"   Symmetric  - Avg MSE: {sym_results['total_mse']/sym_results['layers']:.8f} | Size: {sym_size:.2f} MB")
print(f"   Asymmetric - Avg MSE: {asym_results['total_mse']/asym_results['layers']:.8f} | Size: {asym_size:.2f} MB")
print(f"\n✅ Both versions saved!")
print(f"   Symmetric:  {os.path.abspath(sym_save_path)}")
print(f"   Asymmetric: {os.path.abspath(asym_save_path)}")

del model_for_quant
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 10: BLOCK-WISE QUANTIZATION - Applied to Qwen2-1.5B
# ============================================================================
# Block-wise quantization divides weights into blocks with separate scales

print("\n" + "="*80)
print("🔬 TECHNIQUE 10: BLOCK-WISE QUANTIZATION")
print("="*80)

print("""
📖 How Block-wise Quantization Works:
   
   Traditional: One scale for entire tensor → loses precision
   Block-wise:  Divide into blocks, each block has its own scale
   
   Example (block_size=64):
   [w0...w63] → scale_0    [w64...w127] → scale_1  ...
   
   Benefits:
   - Better precision than per-tensor
   - Less overhead than per-element
   - Used in GPTQ, AWQ, and most modern INT4 methods
""")

def blockwise_quantize(tensor, block_size=64, num_bits=8):
    """
    Block-wise INT8 quantization for Qwen2-1.5B weights
    """
    original_shape = tensor.shape
    flat = tensor.flatten().float()
    
    # Pad to be divisible by block_size
    pad_size = (block_size - len(flat) % block_size) % block_size
    if pad_size > 0:
        flat = torch.nn.functional.pad(flat, (0, pad_size))
    
    # Reshape into blocks
    blocks = flat.reshape(-1, block_size)
    num_blocks = blocks.shape[0]
    
    # Quantize each block
    qmax = 2 ** (num_bits - 1) - 1
    scales = torch.zeros(num_blocks)
    quantized = torch.zeros_like(blocks, dtype=torch.int8)
    
    for i in range(num_blocks):
        block = blocks[i]
        abs_max = torch.max(torch.abs(block))
        scale = abs_max / qmax if abs_max > 0 else 1.0
        scales[i] = scale
        quantized[i] = torch.round(block / scale).clamp(-qmax, qmax).to(torch.int8)
    
    return quantized, scales, original_shape, pad_size

def blockwise_dequantize(quantized, scales, original_shape, pad_size, block_size=64):
    """Dequantize block-wise quantized tensor"""
    num_blocks = quantized.shape[0]
    dequantized = torch.zeros_like(quantized, dtype=torch.float32)
    
    for i in range(num_blocks):
        dequantized[i] = quantized[i].float() * scales[i]
    
    flat = dequantized.flatten()
    if pad_size > 0:
        flat = flat[:-pad_size]
    
    return flat.reshape(original_shape)

# Apply block-wise quantization to Qwen2-1.5B
print("⏳ Loading Qwen2-1.5B for block-wise quantization...")
model_blockwise = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

block_sizes = [32, 64, 128]  # Test different block sizes
blockwise_results = {}

for block_size in block_sizes:
    print(f"\n⏳ Testing block_size={block_size}...")
    total_mse = 0
    total_layers = 0
    quantized_state = {}
    
    for name, param in model_blockwise.named_parameters():
        if param.ndim >= 2 and param.numel() > 1000:
            weight = param.data
            q, scales, shape, pad = blockwise_quantize(weight, block_size=block_size)
            deq = blockwise_dequantize(q, scales, shape, pad, block_size=block_size)
            mse = torch.mean((weight - deq) ** 2).item()
            total_mse += mse
            total_layers += 1
            quantized_state[name] = {"quantized": q, "scales": scales, "shape": shape, "pad": pad}
    
    avg_mse = total_mse / total_layers
    blockwise_results[block_size] = {"avg_mse": avg_mse, "state": quantized_state}
    print(f"   Block size {block_size}: Avg MSE = {avg_mse:.8f}")

# Save the best block size version (usually 64)
best_block_size = 64
blockwise_save_path = f"Qwen2-1.5B-Instruct-INT8-Blockwise-{best_block_size}"
os.makedirs(blockwise_save_path, exist_ok=True)

torch.save(blockwise_results[best_block_size]["state"], 
           os.path.join(blockwise_save_path, f"model_blockwise_int8_b{best_block_size}.pt"))
tokenizer.save_pretrained(blockwise_save_path)
model_blockwise.config.save_pretrained(blockwise_save_path)

saved_size = os.path.getsize(os.path.join(blockwise_save_path, f"model_blockwise_int8_b{best_block_size}.pt")) / (1024**2)

print(f"\n✅ Block-wise Quantization Complete!")
print(f"   Best block size: {best_block_size}")
print(f"   Saved size: {saved_size:.2f} MB")
print(f"   Saved to: {os.path.abspath(blockwise_save_path)}")

del model_blockwise
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 11: ABSMAX QUANTIZATION - Applied to Qwen2-1.5B
# ============================================================================
# AbsMax is the simplest symmetric quantization, used as baseline

print("\n" + "="*80)
print("🔬 TECHNIQUE 11: ABSMAX QUANTIZATION")
print("="*80)

print("""
📖 How AbsMax Quantization Works:
   
   Formula: x_quant = round(x / scale) where scale = max(|x|) / 127
   
   This is the simplest form of symmetric quantization:
   1. Find absolute maximum value in tensor
   2. Scale all values to fit in [-127, 127]
   3. Round to nearest integer
   
   Used as baseline in many papers (LLM.int8(), etc.)
""")

def absmax_quantize_tensor(tensor, bits=8):
    """AbsMax quantization - simplest symmetric method"""
    qmax = 2 ** (bits - 1) - 1  # 127 for 8-bit
    scale = torch.max(torch.abs(tensor)) / qmax
    scale = scale if scale > 0 else torch.tensor(1.0)
    quantized = torch.round(tensor / scale).clamp(-qmax, qmax).to(torch.int8)
    return quantized, scale

def absmax_dequantize(quantized, scale):
    return quantized.float() * scale

# Apply AbsMax to Qwen2-1.5B
print("⏳ Loading Qwen2-1.5B for AbsMax quantization...")
model_absmax = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

absmax_state = {}
total_original_bytes = 0
total_quantized_bytes = 0
total_mse = 0
num_layers = 0

print("⏳ Applying AbsMax quantization to all layers...")
for name, param in model_absmax.named_parameters():
    weight = param.data
    total_original_bytes += weight.numel() * weight.element_size()
    
    if weight.ndim >= 2:  # Quantize weight matrices
        q, scale = absmax_quantize_tensor(weight)
        absmax_state[name] = {"quantized": q, "scale": scale, "shape": weight.shape}
        total_quantized_bytes += q.numel() * 1 + 4  # INT8 + FP32 scale
        
        # Calculate MSE
        deq = absmax_dequantize(q, scale)
        mse = torch.mean((weight - deq) ** 2).item()
        total_mse += mse
        num_layers += 1
    else:
        # Keep biases and small tensors in original format
        absmax_state[name] = {"original": weight}
        total_quantized_bytes += weight.numel() * weight.element_size()

# Save AbsMax quantized model
absmax_save_path = "Qwen2-1.5B-Instruct-INT8-AbsMax"
os.makedirs(absmax_save_path, exist_ok=True)

torch.save(absmax_state, os.path.join(absmax_save_path, "model_absmax_int8.pt"))
tokenizer.save_pretrained(absmax_save_path)
model_absmax.config.save_pretrained(absmax_save_path)

saved_size = os.path.getsize(os.path.join(absmax_save_path, "model_absmax_int8.pt")) / (1024**2)

print(f"\n✅ AbsMax Quantization Complete!")
print(f"   Original size: {total_original_bytes / (1024**2):.2f} MB")
print(f"   Theoretical quantized: {total_quantized_bytes / (1024**2):.2f} MB")
print(f"   Saved file size: {saved_size:.2f} MB")
print(f"   Compression: {total_original_bytes / (saved_size * 1024**2):.2f}x")
print(f"   Average MSE: {total_mse / num_layers:.8f}")
print(f"   Saved to: {os.path.abspath(absmax_save_path)}")

del model_absmax
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 12: MIXED-PRECISION QUANTIZATION - Applied to Qwen2-1.5B
# ============================================================================
# Keep sensitive layers in higher precision, quantize less sensitive ones

print("\n" + "="*80)
print("🔬 TECHNIQUE 12: MIXED-PRECISION QUANTIZATION")
print("="*80)

print("""
📖 How Mixed-Precision Quantization Works:
   
   Not all layers are equally important:
   - Embedding layers: Often kept in FP16 (high impact on output)
   - Attention layers: Sometimes kept in FP16 (QKV projections)
   - FFN layers: Can be aggressively quantized (INT8 or INT4)
   - Output/LM head: Often kept in FP16
   
   Strategy:
   - Small layers (<10K params) → FP16
   - Embedding/Output → FP16
   - Attention (q,k,v,o_proj) → INT8
   - FFN (gate,up,down_proj) → INT8
""")

print("⏳ Loading Qwen2-1.5B for mixed-precision quantization...")
model_mixed = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

mixed_state = {}
layer_stats = {"fp16": 0, "int8": 0, "fp16_size": 0, "int8_size": 0}

# Define which layers to keep in FP16
def should_keep_fp16(name, param):
    # Keep embedding and lm_head in FP16
    if "embed" in name.lower() or "lm_head" in name.lower():
        return True
    # Keep small layers in FP16
    if param.numel() < 10000:
        return True
    # Keep normalization layers in FP16
    if "norm" in name.lower() or "layernorm" in name.lower():
        return True
    return False

print("⏳ Applying mixed-precision quantization...")
for name, param in model_mixed.named_parameters():
    weight = param.data
    
    if should_keep_fp16(name, param):
        # Keep in FP16
        mixed_state[name] = {"dtype": "fp16", "data": weight.half()}
        layer_stats["fp16"] += 1
        layer_stats["fp16_size"] += weight.numel() * 2  # FP16 = 2 bytes
    else:
        # Quantize to INT8
        q, scale = absmax_quantize_tensor(weight)
        mixed_state[name] = {"dtype": "int8", "quantized": q, "scale": scale}
        layer_stats["int8"] += 1
        layer_stats["int8_size"] += q.numel() * 1 + 4  # INT8 + scale

# Save mixed-precision model
mixed_save_path = "Qwen2-1.5B-Instruct-MixedPrecision"
os.makedirs(mixed_save_path, exist_ok=True)

torch.save(mixed_state, os.path.join(mixed_save_path, "model_mixed_precision.pt"))
tokenizer.save_pretrained(mixed_save_path)
model_mixed.config.save_pretrained(mixed_save_path)

saved_size = os.path.getsize(os.path.join(mixed_save_path, "model_mixed_precision.pt")) / (1024**2)

print(f"\n✅ Mixed-Precision Quantization Complete!")
print(f"   Layers in FP16: {layer_stats['fp16']} ({layer_stats['fp16_size'] / (1024**2):.2f} MB)")
print(f"   Layers in INT8: {layer_stats['int8']} ({layer_stats['int8_size'] / (1024**2):.2f} MB)")
print(f"   Saved file size: {saved_size:.2f} MB")
print(f"   Saved to: {os.path.abspath(mixed_save_path)}")

del model_mixed
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 13: K-MEANS WEIGHT CLUSTERING + QUANTIZATION
# ============================================================================
# Cluster weights into K centroids, store only cluster indices

print("\n" + "="*80)
print("🔬 TECHNIQUE 13: K-MEANS WEIGHT CLUSTERING")
print("="*80)

print("""
📖 How K-Means Weight Clustering Works:
   
   Instead of quantizing to fixed intervals, cluster weights to learned centroids:
   1. Run K-Means on weight values (e.g., K=16 for 4-bit)
   2. Replace each weight with its cluster centroid index
   3. Store: cluster indices (log2(K) bits each) + K centroids (FP32)
   
   Benefits:
   - Centroids adapt to actual weight distribution
   - Can achieve better accuracy than uniform quantization
   - Used in "Deep Compression" paper (Han et al., 2016)
""")

from sklearn.cluster import MiniBatchKMeans
import numpy as np

def kmeans_quantize(tensor, n_clusters=16):
    """
    K-Means weight clustering quantization
    n_clusters=16 → 4-bit, n_clusters=256 → 8-bit
    """
    original_shape = tensor.shape
    flat = tensor.flatten().numpy().reshape(-1, 1)
    
    # Use MiniBatchKMeans for large tensors (faster)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=1024, n_init=3)
    labels = kmeans.fit_predict(flat)
    centroids = kmeans.cluster_centers_.flatten()
    
    # Determine index dtype based on n_clusters
    if n_clusters <= 16:
        index_dtype = np.uint8  # 4-bit stored as 8-bit
    elif n_clusters <= 256:
        index_dtype = np.uint8
    else:
        index_dtype = np.uint16
    
    return labels.astype(index_dtype), centroids.astype(np.float32), original_shape

def kmeans_dequantize(labels, centroids, original_shape):
    """Reconstruct tensor from cluster indices and centroids"""
    reconstructed = centroids[labels]
    return torch.tensor(reconstructed.reshape(original_shape))

# Apply K-Means clustering to a sample of Qwen2-1.5B layers
print("⏳ Loading Qwen2-1.5B for K-Means clustering...")
model_kmeans = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Test different cluster sizes
cluster_configs = [
    {"n_clusters": 16, "bits": 4},
    {"n_clusters": 256, "bits": 8}
]

for config in cluster_configs:
    n_clusters = config["n_clusters"]
    bits = config["bits"]
    print(f"\n⏳ Testing K-Means with {n_clusters} clusters ({bits}-bit)...")
    
    kmeans_state = {}
    total_mse = 0
    num_layers = 0
    
    # Only process first few large layers (full model takes too long)
    layers_processed = 0
    max_layers = 10  # Process first 10 large layers for demo
    
    for name, param in model_kmeans.named_parameters():
        if param.ndim >= 2 and param.numel() > 10000 and layers_processed < max_layers:
            weight = param.data
            
            # Subsample for large tensors to speed up
            if weight.numel() > 100000:
                # K-Means on subsample, then apply to all
                sample_size = min(50000, weight.numel())
                flat = weight.flatten()
                indices = torch.randperm(flat.numel())[:sample_size]
                sample = flat[indices].numpy().reshape(-1, 1)
                
                kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=1024, n_init=3)
                kmeans.fit(sample)
                
                # Apply to full tensor
                labels = kmeans.predict(flat.numpy().reshape(-1, 1))
                centroids = kmeans.cluster_centers_.flatten()
            else:
                labels, centroids, _ = kmeans_quantize(weight, n_clusters)
            
            # Calculate MSE
            reconstructed = centroids[labels].reshape(weight.shape)
            mse = np.mean((weight.numpy() - reconstructed) ** 2)
            total_mse += mse
            num_layers += 1
            layers_processed += 1
            
            kmeans_state[name] = {
                "labels": labels.astype(np.uint8 if n_clusters <= 256 else np.uint16),
                "centroids": centroids,
                "shape": weight.shape
            }
    
    if num_layers > 0:
        avg_mse = total_mse / num_layers
        print(f"   Avg MSE ({bits}-bit): {avg_mse:.8f}")
        
        # Save this configuration
        kmeans_save_path = f"Qwen2-1.5B-Instruct-KMeans-{bits}bit"
        os.makedirs(kmeans_save_path, exist_ok=True)
        
        # Save as numpy for efficiency
        np.savez_compressed(
            os.path.join(kmeans_save_path, f"model_kmeans_{bits}bit.npz"),
            **{f"{k}_labels": v["labels"] for k, v in kmeans_state.items()},
            **{f"{k}_centroids": v["centroids"] for k, v in kmeans_state.items()}
        )
        tokenizer.save_pretrained(kmeans_save_path)
        
        saved_size = os.path.getsize(os.path.join(kmeans_save_path, f"model_kmeans_{bits}bit.npz")) / (1024**2)
        print(f"   Saved size: {saved_size:.2f} MB")
        print(f"   Saved to: {os.path.abspath(kmeans_save_path)}")

del model_kmeans
torch.cuda.empty_cache()

print("\n✅ K-Means Clustering Complete!")

In [ ]:
# ============================================================================
# TECHNIQUE 14: MINMAX QUANTIZATION - Applied to Qwen2-1.5B
# ============================================================================
# Uses actual min/max values instead of symmetric range

print("\n" + "="*80)
print("🔬 TECHNIQUE 14: MINMAX QUANTIZATION")
print("="*80)

print("""
📖 How MinMax Quantization Works:
   
   Unlike AbsMax (symmetric), MinMax uses the actual range:
   
   scale = (max - min) / (qmax - qmin)
   zero_point = round(qmin - min / scale)
   x_quant = round(x / scale) + zero_point
   
   Better for:
   - Asymmetric weight distributions
   - Weights that don't center around zero
   - ReLU activations (all positive)
""")

def minmax_quantize(tensor, bits=8):
    """MinMax quantization using actual min/max range"""
    qmin = -(2 ** (bits - 1))
    qmax = 2 ** (bits - 1) - 1
    
    min_val = tensor.min()
    max_val = tensor.max()
    
    scale = (max_val - min_val) / (qmax - qmin)
    scale = scale if scale > 0 else torch.tensor(1.0)
    
    zero_point = qmin - torch.round(min_val / scale)
    zero_point = zero_point.clamp(qmin, qmax).to(torch.int8)
    
    quantized = torch.round(tensor / scale + zero_point).clamp(qmin, qmax).to(torch.int8)
    
    return quantized, scale, zero_point

def minmax_dequantize(quantized, scale, zero_point):
    return (quantized.float() - zero_point) * scale

# Apply MinMax to Qwen2-1.5B
print("⏳ Loading Qwen2-1.5B for MinMax quantization...")
model_minmax = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

minmax_state = {}
total_mse = 0
num_layers = 0

print("⏳ Applying MinMax quantization to all layers...")
for name, param in model_minmax.named_parameters():
    weight = param.data
    
    if weight.ndim >= 2:
        q, scale, zp = minmax_quantize(weight)
        minmax_state[name] = {"quantized": q, "scale": scale, "zero_point": zp, "shape": weight.shape}
        
        # Calculate MSE
        deq = minmax_dequantize(q, scale, zp)
        mse = torch.mean((weight - deq) ** 2).item()
        total_mse += mse
        num_layers += 1
    else:
        minmax_state[name] = {"original": weight}

# Save MinMax quantized model
minmax_save_path = "Qwen2-1.5B-Instruct-INT8-MinMax"
os.makedirs(minmax_save_path, exist_ok=True)

torch.save(minmax_state, os.path.join(minmax_save_path, "model_minmax_int8.pt"))
tokenizer.save_pretrained(minmax_save_path)
model_minmax.config.save_pretrained(minmax_save_path)

saved_size = os.path.getsize(os.path.join(minmax_save_path, "model_minmax_int8.pt")) / (1024**2)

print(f"\n✅ MinMax Quantization Complete!")
print(f"   Average MSE: {total_mse / num_layers:.8f}")
print(f"   Saved file size: {saved_size:.2f} MB")
print(f"   Saved to: {os.path.abspath(minmax_save_path)}")

del model_minmax
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# TECHNIQUE 15: HISTOGRAM-BASED QUANTIZATION (Percentile Clipping)
# ============================================================================
# Clip outliers before quantization for better accuracy

print("\n" + "="*80)
print("🔬 TECHNIQUE 15: HISTOGRAM-BASED QUANTIZATION (Percentile Clipping)")
print("="*80)

print("""
📖 How Histogram/Percentile Quantization Works:
   
   Problem: Outliers waste quantization range
   Solution: Clip values at percentiles (e.g., 99.9%)
   
   Steps:
   1. Compute histogram of weight values
   2. Find percentile thresholds (e.g., 0.1% and 99.9%)
   3. Clip values outside thresholds
   4. Quantize the clipped range
   
   Benefits:
   - Better utilization of quantization bins
   - Outliers don't dominate the scale
   - Used in TensorRT and many production systems
""")

def histogram_quantize(tensor, bits=8, percentile=99.9):
    """Histogram-based quantization with percentile clipping"""
    qmin = -(2 ** (bits - 1))
    qmax = 2 ** (bits - 1) - 1
    
    # Find percentile thresholds
    flat = tensor.flatten()
    low_thresh = torch.quantile(flat, (100 - percentile) / 100)
    high_thresh = torch.quantile(flat, percentile / 100)
    
    # Clip tensor
    clipped = torch.clamp(tensor, low_thresh, high_thresh)
    
    # Quantize clipped tensor
    scale = (high_thresh - low_thresh) / (qmax - qmin)
    scale = scale if scale > 0 else torch.tensor(1.0)
    
    zero_point = qmin - torch.round(low_thresh / scale)
    quantized = torch.round(clipped / scale + zero_point).clamp(qmin, qmax).to(torch.int8)
    
    return quantized, scale, zero_point, low_thresh, high_thresh

def histogram_dequantize(quantized, scale, zero_point):
    return (quantized.float() - zero_point) * scale

# Apply Histogram quantization to Qwen2-1.5B
print("⏳ Loading Qwen2-1.5B for Histogram quantization...")
model_hist = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map={"": "cpu"},
    low_cpu_mem_usage=True
)

# Test different percentiles
percentiles = [99.0, 99.9, 99.99]
best_percentile = 99.9
best_mse = float('inf')

for pct in percentiles:
    total_mse = 0
    num_layers = 0
    
    for name, param in model_hist.named_parameters():
        if param.ndim >= 2 and param.numel() > 1000:
            weight = param.data
            q, scale, zp, low, high = histogram_quantize(weight, percentile=pct)
            deq = histogram_dequantize(q, scale, zp)
            mse = torch.mean((weight - deq) ** 2).item()
            total_mse += mse
            num_layers += 1
    
    avg_mse = total_mse / num_layers
    print(f"   Percentile {pct}%: Avg MSE = {avg_mse:.8f}")
    
    if avg_mse < best_mse:
        best_mse = avg_mse
        best_percentile = pct

# Save with best percentile
print(f"\n⏳ Saving with best percentile ({best_percentile}%)...")
hist_state = {}

for name, param in model_hist.named_parameters():
    weight = param.data
    if weight.ndim >= 2:
        q, scale, zp, low, high = histogram_quantize(weight, percentile=best_percentile)
        hist_state[name] = {"quantized": q, "scale": scale, "zero_point": zp, 
                           "low_thresh": low, "high_thresh": high}
    else:
        hist_state[name] = {"original": weight}

hist_save_path = "Qwen2-1.5B-Instruct-INT8-Histogram"
os.makedirs(hist_save_path, exist_ok=True)

torch.save(hist_state, os.path.join(hist_save_path, "model_histogram_int8.pt"))
tokenizer.save_pretrained(hist_save_path)
model_hist.config.save_pretrained(hist_save_path)

saved_size = os.path.getsize(os.path.join(hist_save_path, "model_histogram_int8.pt")) / (1024**2)

print(f"\n✅ Histogram Quantization Complete!")
print(f"   Best percentile: {best_percentile}%")
print(f"   Best MSE: {best_mse:.8f}")
print(f"   Saved size: {saved_size:.2f} MB")
print(f"   Saved to: {os.path.abspath(hist_save_path)}")

del model_hist
torch.cuda.empty_cache()

In [ ]:
# ============================================================================
# FINAL MASTER SUMMARY: ALL 15 QUANTIZATION TECHNIQUES
# ============================================================================

print("\n" + "="*80)
print("🏆 FINAL MASTER SUMMARY: ALL QUANTIZATION TECHNIQUES FOR QWEN2-1.5B")
print("="*80)

# Collect all saved models
all_saved_models = []

model_paths = [
    ("Dynamic INT8", "Qwen2-1.5B-Instruct-INT8", "pytorch_model_int8.bin"),
    ("FP16", "Qwen2-1.5B-Instruct-FP16", "pytorch_model_fp16.bin"),
    ("BF16", "Qwen2-1.5B-Instruct-BF16", "pytorch_model_bf16.bin"),
    ("Symmetric INT8", "Qwen2-1.5B-Instruct-INT8-Symmetric", "model_symmetric_int8.pt"),
    ("Asymmetric INT8", "Qwen2-1.5B-Instruct-INT8-Asymmetric", "model_asymmetric_int8.pt"),
    ("Block-wise INT8", "Qwen2-1.5B-Instruct-INT8-Blockwise-64", "model_blockwise_int8_b64.pt"),
    ("AbsMax INT8", "Qwen2-1.5B-Instruct-INT8-AbsMax", "model_absmax_int8.pt"),
    ("Mixed Precision", "Qwen2-1.5B-Instruct-MixedPrecision", "model_mixed_precision.pt"),
    ("K-Means 4-bit", "Qwen2-1.5B-Instruct-KMeans-4bit", "model_kmeans_4bit.npz"),
    ("K-Means 8-bit", "Qwen2-1.5B-Instruct-KMeans-8bit", "model_kmeans_8bit.npz"),
    ("MinMax INT8", "Qwen2-1.5B-Instruct-INT8-MinMax", "model_minmax_int8.pt"),
    ("Histogram INT8", "Qwen2-1.5B-Instruct-INT8-Histogram", "model_histogram_int8.pt"),
    ("Static INT8", "static_int8_model", "model_static_int8.pt"),
    ("ONNX INT8", "onnx_quantized_model", "model_int8.onnx"),
    ("Pruned + INT8", "pruned_quantized_model", "model_pruned_int8.pt"),
    ("Per-Channel INT8", "per_channel_quantized_model", "model_per_channel_int8.pt"),
    ("QAT INT8", "qat_quantized_model", "model_qat_int8.pt"),
]

print("\n📋 SAVED QUANTIZED MODELS:")
print("-" * 100)
print(f"{'#':<3} {'Technique':<25} {'Path':<45} {'Size':<15}")
print("-" * 100)

idx = 1
for name, path, filename in model_paths:
    filepath = os.path.join(path, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        if size > 1024**2:
            size_str = f"{size / (1024**2):.2f} MB"
        else:
            size_str = f"{size / 1024:.2f} KB"
        print(f"{idx:<3} {name:<25} {path:<45} {size_str:<15}")
        all_saved_models.append({"name": name, "path": path, "size": size_str})
        idx += 1

print("-" * 100)
print(f"\n✅ Total techniques implemented: {len(all_saved_models)}")

# Save comprehensive summary
comprehensive_summary = {
    "project": "LLM Quantization - Qwen2-1.5B-Instruct",
    "model_id": "Qwen/Qwen2-1.5B-Instruct",
    "total_techniques": len(all_saved_models),
    "techniques": {
        "precision_reduction": ["FP16", "BF16"],
        "integer_quantization": ["Dynamic INT8", "Static INT8", "Symmetric INT8", 
                                 "Asymmetric INT8", "AbsMax INT8", "MinMax INT8"],
        "advanced_quantization": ["Block-wise INT8", "Per-Channel INT8", "Histogram INT8"],
        "clustering_based": ["K-Means 4-bit", "K-Means 8-bit"],
        "training_based": ["QAT INT8"],
        "combined": ["Mixed Precision", "Pruning + INT8"],
        "export_formats": ["ONNX INT8"]
    },
    "saved_models": all_saved_models,
    "technique_explanations": {
        "FP16": "Convert 32-bit floats to 16-bit, 2x compression",
        "BF16": "Brain Float16 - same range as FP32, less precision, good for training",
        "Dynamic INT8": "Weights quantized ahead, activations quantized at runtime",
        "Static INT8": "Both weights and activations quantized using calibration",
        "Symmetric INT8": "Zero-centered quantization, scale = max(|x|) / 127",
        "Asymmetric INT8": "Uses full INT8 range with zero-point offset",
        "AbsMax INT8": "Simplest symmetric method, scale = absmax / 127",
        "MinMax INT8": "Uses actual min/max range for better accuracy",
        "Block-wise INT8": "Separate scale per block (e.g., 64 weights)",
        "Per-Channel INT8": "Separate scale per output channel",
        "Histogram INT8": "Clips outliers at percentiles before quantization",
        "K-Means": "Cluster weights to K centroids, store indices",
        "Mixed Precision": "Keep sensitive layers in FP16, quantize others",
        "QAT": "Simulate quantization during training for robustness",
        "Pruning + INT8": "Remove small weights (sparsity) + quantization",
        "ONNX INT8": "Export to ONNX format with INT8 for portability"
    }
}

with open("comprehensive_quantization_summary.json", "w", encoding="utf-8") as f:
    json.dump(comprehensive_summary, f, indent=2, ensure_ascii=False)

print(f"\n📄 Comprehensive summary saved to 'comprehensive_quantization_summary.json'")

# Print technique comparison table
print("\n" + "="*80)
print("📊 TECHNIQUE COMPARISON:")
print("="*80)
print("""
┌─────────────────────────┬─────────────┬──────────────┬───────────────┬──────────────┐
│ Technique               │ Compression │ Quality Loss │ Speed Impact  │ Best For     │
├─────────────────────────┼─────────────┼──────────────┼───────────────┼──────────────┤
│ FP16                    │ 2x          │ < 0.1%       │ Faster (GPU)  │ Default      │
│ BF16                    │ 2x          │ < 0.5%       │ Similar       │ Training     │
│ Dynamic INT8            │ 4x          │ ~1%          │ Slower        │ CPU deploy   │
│ Static INT8             │ 4x          │ ~1%          │ Faster        │ Production   │
│ Symmetric INT8          │ 4x          │ ~1%          │ Fastest       │ Simple use   │
│ Asymmetric INT8         │ 4x          │ < 1%         │ Fast          │ Better acc   │
│ Block-wise INT8         │ 4x          │ < 1%         │ Medium        │ LLMs         │
│ Per-Channel INT8        │ 4x          │ < 0.5%       │ Medium        │ Conv layers  │
│ Histogram INT8          │ 4x          │ < 1%         │ Medium        │ Outliers     │
│ K-Means 4-bit           │ 8x          │ ~2-5%        │ Slow          │ Max compress │
│ K-Means 8-bit           │ 4x          │ ~1%          │ Medium        │ Adaptive     │
│ Mixed Precision         │ 2-4x        │ < 1%         │ Variable      │ Accuracy     │
│ QAT INT8                │ 4x          │ < 0.5%       │ Fast          │ Best quality │
│ Pruning + INT8          │ 5-8x        │ ~2-5%        │ Fast          │ Max compress │
│ ONNX INT8               │ 4x          │ ~1%          │ Fast          │ Cross-plat   │
└─────────────────────────┴─────────────┴──────────────┴───────────────┴──────────────┘
""")

In [8]:
# Save the quantized model
save_path = "Qwen2-1.5B-Instruct-INT8"

print(f"💾 Saving quantized model to '{save_path}'...")

# Create directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the quantized model state dict
torch.save(model_int8.state_dict(), os.path.join(save_path, "pytorch_model_int8.bin"))

# Save the tokenizer
tokenizer.save_pretrained(save_path)

# Save model config
model_int8.config.save_pretrained(save_path)

# Calculate saved model file size
saved_model_path = os.path.join(save_path, "pytorch_model_int8.bin")
saved_size_mb = os.path.getsize(saved_model_path) / (1024**2)

print(f"✅ Quantized model saved!")
print(f"📂 Location: {os.path.abspath(save_path)}")
print(f"📏 Saved model file size: {saved_size_mb:.2f} MB")

💾 Saving quantized model to 'Qwen2-1.5B-Instruct-INT8'...
✅ Quantized model saved!
📂 Location: c:\Users\LENOVO\Documents\Assignment test\Qwen2-1.5B-Instruct-INT8
📏 Saved model file size: 2363.14 MB


In [9]:
# Use the complete save function to add model card and loader script
model_info = save_quantized_model_complete(
    model=model_int8,
    tokenizer=tokenizer,
    save_path="Qwen2-1.5B-Instruct-INT8",
    model_id=model_id,
    system_specs=system_specs
)

print("\n" + "="*60)
print("📋 MODEL CARD SUMMARY")
print("="*60)
for key, value in model_info.items():
    if isinstance(value, dict):
        print(f"\n{key}:")
        for k, v in value.items():
            print(f"   {k}: {v}")
    else:
        print(f"{key}: {value}")

💾 Saving quantized model to 'Qwen2-1.5B-Instruct-INT8'...
   ✅ Model weights saved
   ✅ Tokenizer saved
   ✅ Model config saved
   ✅ Model card saved
   ✅ Loader script saved

✅ Model completely saved to: c:\Users\LENOVO\Documents\Assignment test\Qwen2-1.5B-Instruct-INT8
📏 Total model size: 2363.14 MB
🍓 Raspberry Pi compatible: ✅ Yes

📋 MODEL CARD SUMMARY
model_id: Qwen/Qwen2-1.5B-Instruct
quantization: PyTorch Dynamic INT8
model_file: pytorch_model_int8.bin
model_size_mb: 2363.14

minimum_requirements:
   ram_gb: 4.6
   storage_gb: 3.3
   python_version: 3.8+
   pytorch_version: 1.9+

compatible_devices:
   raspberry_pi_5_8gb: True
   raspberry_pi_4_8gb: True
   raspberry_pi_4_4gb: False
   jetson_nano: True
   orange_pi_5: True
   desktop_8gb_ram: True

usage_instructions:
   load_command: torch.load('pytorch_model_int8.bin')
   inference_device: cpu
   expected_tokens_per_sec: 1-5 on Raspberry Pi, 5-15 on desktop

created_on:
   os: Windows 10
   processor: AMD64 Family 23 Model 17 

# 🍓 Device Compatibility & Requirements

## Minimum Hardware Requirements for INT8 Quantized Model

| Device | RAM Required | Storage | CPU | Recommended |
|--------|-------------|---------|-----|-------------|
| **Raspberry Pi 5** | 8GB | 4GB+ | ARM Cortex-A76 | ✅ Yes |
| **Raspberry Pi 4** | 8GB | 4GB+ | ARM Cortex-A72 | ⚠️ Slow but works |
| **Raspberry Pi 4** | 4GB | 4GB+ | ARM Cortex-A72 | ❌ Not enough RAM |
| **Jetson Nano** | 4GB | 4GB+ | ARM + GPU | ⚠️ Use GPU instead |
| **Orange Pi 5** | 8GB+ | 4GB+ | RK3588 | ✅ Yes |
| **Desktop/Laptop** | 8GB+ | 4GB+ | x86_64 | ✅ Yes |

## Model Size Summary
- **FP32 (Original)**: ~6 GB RAM needed
- **FP16 (Half Precision)**: ~3 GB RAM needed  
- **INT8 (Quantized)**: ~1.5-2 GB RAM needed
- **INT4 (Further Quantized)**: ~0.8-1 GB RAM needed

## For Raspberry Pi Deployment
```bash
# Install dependencies on Raspberry Pi
pip install torch --index-url https://download.pytorch.org/whl/cpu
pip install transformers accelerate
```

In [2]:
def save_quantized_model_complete(model, tokenizer, save_path, model_id, system_specs=None):
    """
    Complete function to save a quantized model with all necessary files.
    
    Args:
        model: The quantized PyTorch model
        tokenizer: The tokenizer
        save_path: Directory path to save the model
        model_id: Original model identifier
        system_specs: Optional system specifications dict
    
    Returns:
        dict: Information about the saved model
    """
    import os
    import json
    import torch
    
    print(f"💾 Saving quantized model to '{save_path}'...")
    
    # Create directory
    os.makedirs(save_path, exist_ok=True)
    
    # 1. Save model weights
    model_file = os.path.join(save_path, "pytorch_model_int8.bin")
    torch.save(model.state_dict(), model_file)
    print(f"   ✅ Model weights saved")
    
    # 2. Save tokenizer
    tokenizer.save_pretrained(save_path)
    print(f"   ✅ Tokenizer saved")
    
    # 3. Save model config
    model.config.save_pretrained(save_path)
    print(f"   ✅ Model config saved")
    
    # 4. Calculate sizes
    model_size_mb = os.path.getsize(model_file) / (1024**2)
    
    # 5. Create model card with device requirements
    model_card = {
        "model_id": model_id,
        "quantization": "PyTorch Dynamic INT8",
        "model_file": "pytorch_model_int8.bin",
        "model_size_mb": round(model_size_mb, 2),
        "minimum_requirements": {
            "ram_gb": max(2, round(model_size_mb / 1024 * 2, 1)),  # 2x model size for safety
            "storage_gb": round(model_size_mb / 1024 + 1, 1),  # Model + overhead
            "python_version": "3.8+",
            "pytorch_version": "1.9+"
        },
        "compatible_devices": {
            "raspberry_pi_5_8gb": True,
            "raspberry_pi_4_8gb": True,
            "raspberry_pi_4_4gb": model_size_mb < 2000,  # Only if model < 2GB
            "jetson_nano": True,
            "orange_pi_5": True,
            "desktop_8gb_ram": True
        },
        "usage_instructions": {
            "load_command": "torch.load('pytorch_model_int8.bin')",
            "inference_device": "cpu",
            "expected_tokens_per_sec": "1-5 on Raspberry Pi, 5-15 on desktop"
        }
    }
    
    if system_specs:
        model_card["created_on"] = system_specs
    
    # Save model card
    card_path = os.path.join(save_path, "model_card.json")
    with open(card_path, "w", encoding="utf-8") as f:
        json.dump(model_card, f, indent=2, ensure_ascii=False)
    print(f"   ✅ Model card saved")
    
    # 6. Create a simple loader script
    loader_script = '''"""
Loader script for INT8 quantized model
Run on Raspberry Pi or any CPU device
"""
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_PATH = "."  # Current directory
MODEL_ID = "{model_id}"

def load_quantized_model():
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
    # Load base model architecture
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float32,
        device_map={{"": "cpu"}},
        low_cpu_mem_usage=True
    )
    
    # Apply quantization structure
    model = torch.quantization.quantize_dynamic(
        model, {{torch.nn.Linear}}, dtype=torch.qint8
    )
    
    # Load saved weights
    state_dict = torch.load("pytorch_model_int8.bin", map_location="cpu")
    model.load_state_dict(state_dict)
    
    return model, tokenizer

def generate(model, tokenizer, prompt, max_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=max_tokens,
            do_sample=True, temperature=0.7, top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

if __name__ == "__main__":
    print("Loading quantized model...")
    model, tokenizer = load_quantized_model()
    print("Model loaded! Enter 'quit' to exit.")
    
    while True:
        prompt = input("You: ")
        if prompt.lower() == 'quit':
            break
        response = generate(model, tokenizer, prompt)
        print(f"AI: {{response}}")
'''.format(model_id=model_id)
    
    loader_path = os.path.join(save_path, "load_and_chat.py")
    with open(loader_path, "w", encoding="utf-8") as f:
        f.write(loader_script)
    print(f"   ✅ Loader script saved")
    
    print(f"\n✅ Model completely saved to: {os.path.abspath(save_path)}")
    print(f"📏 Total model size: {model_size_mb:.2f} MB")
    print(f"🍓 Raspberry Pi compatible: {'✅ Yes' if model_size_mb < 3000 else '⚠️ May be slow'}")
    
    return model_card

# Display the function is ready
print("✅ Function 'save_quantized_model_complete' defined and ready to use!")
print("\nUsage:")
print("  model_info = save_quantized_model_complete(model_int8, tokenizer, 'my_model_path', model_id)")
print("\nThis function saves:")
print("  - pytorch_model_int8.bin (model weights)")
print("  - tokenizer files")
print("  - config.json")
print("  - model_card.json (with device compatibility info)")
print("  - load_and_chat.py (ready-to-use script)")

✅ Function 'save_quantized_model_complete' defined and ready to use!

Usage:
  model_info = save_quantized_model_complete(model_int8, tokenizer, 'my_model_path', model_id)

This function saves:
  - pytorch_model_int8.bin (model weights)
  - tokenizer files
  - config.json
  - model_card.json (with device compatibility info)
  - load_and_chat.py (ready-to-use script)


In [11]:
def bigram_similarity(text1, text2):
    """
    Calculate bigram (2-gram) cosine similarity between two texts.
    
    Why bigram similarity can be low:
    - LLMs generate text stochastically (using sampling with temperature/top_p)
    - Even with same prompt, different runs produce different word sequences
    - Bigram similarity compares consecutive word pairs, not semantic meaning
    - Small vocabulary overlap = low similarity even if meaning is similar
    - Different phrasing of same concept = low bigram overlap
    """
    vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'\b\w+\b', min_df=1)
    try:
        X = vectorizer.fit_transform([text1, text2])
        sim = cosine_similarity(X[0], X[1]).item()
        return round(sim, 3)
    except:
        return 0.0

print("\n" + "="*80)
print("📊 QUANTIZATION COMPARISON REPORT")
print("="*80)


all_results = []
for base, quant in zip(baseline_results, quant_results):
    sim = bigram_similarity(base["text"], quant["text"])
    speedup = base["latency"] / quant["latency"] if quant["latency"] > 0 else 0

    print(f"\n{'='*80}")
    print(f"🔸 Prompt {base['id']}: {test_prompts[base['id']-1]['prompt']}")
    print(f"{'='*80}")
    
    print(f"\n📝 FP16 Response:")
    print(f"   {base['text'][:500]}{'...' if len(base['text']) > 500 else ''}")
    print(f"   ⏱️ Latency: {base['latency']}s | Tokens/sec: {base['tokens_per_sec']}")
    
    print(f"\n📝 INT8 Response:")
    print(f"   {quant['text'][:500]}{'...' if len(quant['text']) > 500 else ''}")
    print(f"   ⏱️ Latency: {quant['latency']}s | Tokens/sec: {quant['tokens_per_sec']}")
    
    print(f"\n📊 Metrics:")
    print(f"   Speedup: {speedup:.2f}x | Bigram Similarity: {sim}")

    all_results.append({
        "id": base["id"],
        "prompt": test_prompts[base["id"]-1]["prompt"],
        "fp16_response": base["text"],
        "int8_response": quant["text"],
        "fp16_latency": base["latency"],
        "fp16_tokens_per_sec": base["tokens_per_sec"],
        "int8_latency": quant["latency"],
        "int8_tokens_per_sec": quant["tokens_per_sec"],
        "bigram_similarity": sim,
        "speedup": round(speedup, 2)
    })


📊 QUANTIZATION COMPARISON REPORT

🔸 Prompt 1: Explain how neural networks learn.

📝 FP16 Response:
   Neural networks, also known as artificial neural networks (ANNs), are a type of machine learning algorithm that are inspired by the structure and function of the human brain. They consist of interconnected nodes or units called neurons that process input data through multiple layers of computation.

When training an ANN, it learns to make predictions based on patterns in the training data. This is achieved through a process called backpropagation
   ⏱️ Latency: 438.287s | Tokens/sec: 0.18

📝 INT8 Response:
   The process of learning in neural networks involves the use of backpropagation and optimization algorithms such as stochastic gradient descent to minimize the error between predicted outputs and actual outputs.
Neural networks are trained using a set of labeled examples, where each example consists of input data and its corresponding output label. During training, the network is 

In [ ]:
report = {
    "model": "Qwen/Qwen2-1.5B-Instruct",
    "quantization_methods": ["FP16 Baseline", "PyTorch Dynamic INT8", "INT4 (Theoretical)"],
    "system_specifications": system_specs,
    "model_sizes": {
        "fp16_size_mb": fp16_size_mb,
        "fp32_size_mb": fp32_size_mb,
        "int8_size_mb": int8_size_mb,
        "int4_size_mb": round(int4_size_mb, 2),
        "compression_ratio_int8": round(compression_ratio, 2),
        "compression_ratio_int4": round(fp32_size_mb / int4_size_mb, 2) if int4_size_mb > 0 else 0,
        "memory_saved_int8_mb": round(fp32_size_mb - int8_size_mb, 2),
        "memory_saved_int4_mb": round(fp32_size_mb - int4_size_mb, 2),
        "total_parameters": fp16_params,
        "saved_model_path": os.path.abspath(save_path),
        "saved_model_file_size_mb": round(saved_size_mb, 2)
    },
    "quantization_explained": {
        "INT8": {
            "description": "Dynamic quantization converting FP32 weights to 8-bit integers",
            "how_it_works": [
                "1. Analyze weight distributions (min/max values)",
                "2. Calculate scale factor: scale = (max - min) / 255",
                "3. Quantize: x_int8 = round((x - min) / scale)",
                "4. Store scale/zero-point for dequantization during inference"
            ],
            "compression": "4x vs FP32",
            "quality_loss": "~1%"
        },
        "INT4": {
            "description": "4-bit quantization using per-group scaling",
            "how_it_works": [
                "1. Group weights into blocks (32-128 elements)",
                "2. Calculate per-group scale and zero-point",
                "3. Quantize to 4-bit range [0, 15] or [-8, 7]",
                "4. Pack two INT4 values into one INT8 byte",
                "5. Store group-wise scales for dequantization"
            ],
            "compression": "8x vs FP32",
            "quality_loss": "~2-5%"
        }
    },
    "device_fp16": "CPU",
    "device_int8": "CPU",
    "device_int4": "GPU (BitsAndBytes) or CPU (GGUF format)",
    "note_on_bigram_similarity": (
        "Bigram similarity may be low because LLM generation is stochastic. "
        "Each inference run uses sampling (temperature, top_p) which produces different word sequences. "
        "Bigram similarity measures consecutive word-pair overlap, not semantic meaning. "
        "Two responses can convey the same information with completely different phrasing, "
        "resulting in low bigram similarity but high semantic equivalence."
    ),
    "test_cases": all_results
}

with open("llm_quantization_report.json", "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print("\n✅ Full report saved to 'llm_quantization_report.json'")
print("\n📋 Report includes:")
print("   - System specifications (OS, CPU, RAM, GPU)")
print("   - Model sizes (FP16, FP32, INT8, INT4) and compression ratios")
print("   - Detailed explanation of how INT8 and INT4 quantization work")
print("   - Saved quantized model location")
print("   - Full generated responses for FP16 and INT8")
print("   - Latency and throughput metrics")
print("   - Bigram similarity scores with explanation")
print("\n📖 See 'rapport.md' for comprehensive quantization techniques documentation")


✅ Full report saved to 'llm_quantization_report.json'

📋 Report includes:
   - System specifications (OS, CPU, RAM, GPU)
   - Model sizes (FP16, FP32, INT8) and compression ratio
   - Saved quantized model location
   - Full generated responses for FP16 and INT8
   - Latency and throughput metrics
   - Bigram similarity scores with explanation
